<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20/%D0%90%D0%B9%D0%A2%D0%B8%D0%9E%D0%BD_%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%BE%D0%B2/11_Recognition_Speech__To_Replace__Text_Filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключение библиотек

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install dateparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
import spacy
import dateparser
from datetime import datetime, date
import os
import json
import random
from spacy.training.example import Example
import difflib

Подключаем гуглдиск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. СОЗДАНИЕ БАЗЫ ДЛЯ ОБУЧЕНИЯ СУЩНОСТЕЙ

НАИМЕНОВАНИЕ СОЗДАВАЕМЫХ И ДООБУЧАЕМЫХ СУЩНОСТЕЙ

In [ ]:
#LOC - Местоположение, используется для определения городов отправления и прибытия, а также городов пересадки.
#DATE - Дата, применяется для установления дат отправления и прибытия.
#TIME - Время, используется для указания времени отправления и прибытия.
#TRANSIT - Пересадки, отмечает количество пересадок или их отсутствие.
#AMOUNT - Количество, обозначает количество пассажиров.
#TRANSPORT - Вид транспорта, указывает на тип транспортного средства, например, поезд, самолет, автобус, паром.
#ORG - Организация, относится к компании-перевозчику.
#ANIMAL - Животные, информирует о наличии или отсутствии животных с пассажиром.
#CHILD - Дети, указывает на количество детей или их отсутствие.
#BAGG - Багаж, обозначает наличие или отсутствие багажа.
#LEVEL - Класс обслуживания, например, бизнес, эконом, купе, плацкарт.
#COMFORT - Комфорт, содержит список дополнительных пожеланий, таких как женское купе, нижняя полка и т.д.
#INSUR - Страховка, информирует о наличии или отсутствии страховки.
#BUDGET - Бюджет, указывает общий бюджет на весь маршрут.
#RETURN - Возврат, используется для обозначения обратной поездки.
#FROM и TO - Используются для уточнения направления движения (откуда и куда соответственно).
#ARRIVAL - Прибытие, указывает на дату прибытия.
#GAP - Промежуток, может использоваться для указания на даты или другие временные интервалы.

ФУНКЦИЯ ВЫВОДА ДАННЫХ О ПРЕДЛОЖЕНИИ

In [ ]:
import string

def print_word_indices(sentence):
    punctuation_marks = string.punctuation
    words = sentence.split()
    start_index = 0
    for word in words:
        # Находим индексы начальной и конечной буквы слова
        start_index = sentence.find(word, start_index)
        end_index = start_index + len(word)
        # Если последний символ - знак пунктуации, сдвигаем индекс
        while sentence[end_index - 1] in punctuation_marks:
            end_index -= 1
        print(word, ':', start_index, '-', end_index)
        # Устанавливаем новый стартовый индекс для поиска
        start_index += len(word)

СОБИРАЕМ БАЗУ ДЛЯ ОБУЧЕНИЯ

на основе прошлого моего ноутбука №9 занятия стажировки: транскрибация whisper и реальный текст, размечаем и индексируем, выделянм сущности для предобучения модели на основе "НАИМЕНОВАНИЕ СОЗДАВАЕМЫХ И ДООБУЧАЕМЫХ СУЩНОСТЕЙ" , смотрите выше. для этого, индексируем текст с помощью print_word_indices и затем обрабатываем каждый файл(создаем базу сущностей).

In [ ]:
# файл 001 whisper
("так еду 23 ноября и мне нужно нижнее место не у туалета, где-нибудь посередине вагона",
{"entities": [
    (8, 17, "DATE"),  # 23 ноября
    (30, 85, "COMFORT"),  # нижнее место не у туалета, где-нибудь посередине вагона
]})

('так еду 23 ноября и мне нужно нижнее место не у туалета, где-нибудь посередине вагона',
 {'entities': [(8, 17, 'DATE'), (30, 85, 'COMFORT')]})

In [ ]:
print_word_indices('так еду 23 ноября и мне нужно нижнее место не у туалета, где-нибудь посередине вагона')

так : 0 - 3
еду : 4 - 7
23 : 8 - 10
ноября : 11 - 17
и : 18 - 19
мне : 20 - 23
нужно : 24 - 29
нижнее : 30 - 36
место : 37 - 42
не : 43 - 45
у : 46 - 47
туалета, : 48 - 55
где-нибудь : 57 - 67
посередине : 68 - 78
вагона : 79 - 85


In [ ]:
# файл 002 whisper
("Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.",
{"entities": [
    (2, 4, "FROM"),  # из
    (5, 8, "LOC"),  # Уфы
    (16, 17, "TO"),  # в
    (18, 26, "LOC"),  # Кострому
    (44, 55, "DATE"),  # через неделю
    (64, 74, "DATE"),  # на выходные
    (106, 121, "BUDGET"),   # 10.000 руб
]})

('Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.',
 {'entities': [(2, 4, 'FROM'),
   (5, 8, 'LOC'),
   (16, 17, 'TO'),
   (18, 26, 'LOC'),
   (44, 55, 'DATE'),
   (64, 74, 'DATE'),
   (106, 121, 'BUDGET')]})

In [ ]:
print_word_indices('Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей')

Я : 0 - 1
из : 2 - 4
Уфы, : 5 - 8
поеду : 10 - 15
в : 16 - 17
Кострому. : 18 - 26
Мне : 28 - 31
нужен : 32 - 37
билет : 38 - 43
через : 44 - 49
неделю. : 50 - 56
Лучше : 58 - 63
на : 64 - 66
выходные. : 67 - 75
Так, : 77 - 80
у : 82 - 83
меня : 84 - 88
будет : 89 - 94
на : 95 - 97
поездку : 98 - 105
10 : 106 - 108
тысяч : 109 - 114
рублей : 115 - 121


In [ ]:
# файл 003 whisper
("Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечером мы выезжаем обратно в Москву, и через три дня мы едем в Краснодар. Также пять человек. Найди самый быстрый билет за самое быстрое время.",
{"entities": [
    (5, 11, "DATE"),  # завтра
    (27, 39, "AMOUNT"), #пять человек
    (40, 42, "FROM"),  # из
    (43, 49, "LOC"),  # Москвы
    (50, 51, "TO"),  # в
    (52, 61, "LOC"),  # Петербург
    (63, 70, "DATE"),  # вечером
    (83, 90, "RETURN"),  # обратно
    (91, 92, "TO"),  # в
    (93, 99, "LOC"),  # Москву
    (103, 116, "DATE"),  # через три дня
    (125, 126, "TO"),  # в
    (127, 136, "LOC"),  # Краснодар
    (144, 156, "AMOUNT") # пять человек
]})

('Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечером мы выезжаем обратно в Москву, и через три дня мы едем в Краснодар. Также пять человек. Найди самый быстрый билет за самое быстрое время.',
 {'entities': [(5, 11, 'DATE'),
   (27, 39, 'AMOUNT'),
   (40, 42, 'FROM'),
   (43, 49, 'LOC'),
   (50, 51, 'TO'),
   (52, 61, 'LOC'),
   (63, 70, 'DATE'),
   (83, 90, 'RETURN'),
   (91, 92, 'TO'),
   (93, 99, 'LOC'),
   (103, 116, 'DATE'),
   (125, 126, 'TO'),
   (127, 136, 'LOC'),
   (144, 156, 'AMOUNT')]})

In [ ]:
print_word_indices('Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечером мы выезжаем обратно в Москву, и через три дня мы едем в Краснодар. Также пять человек. Найди самый быстрый билет за самое быстрое время.')

Так, : 0 - 3
завтра : 5 - 11
утром : 12 - 17
мы : 18 - 20
едем, : 21 - 25
пять : 27 - 31
человек : 32 - 39
из : 40 - 42
Москвы : 43 - 49
в : 50 - 51
Петербург. : 52 - 61
Вечером : 63 - 70
мы : 71 - 73
выезжаем : 74 - 82
обратно : 83 - 90
в : 91 - 92
Москву, : 93 - 99
и : 101 - 102
через : 103 - 108
три : 109 - 112
дня : 113 - 116
мы : 117 - 119
едем : 120 - 124
в : 125 - 126
Краснодар. : 127 - 136
Также : 138 - 143
пять : 144 - 148
человек. : 149 - 156
Найди : 158 - 163
самый : 164 - 169
быстрый : 170 - 177
билет : 178 - 183
за : 184 - 186
самое : 187 - 192
быстрое : 193 - 200
время. : 201 - 206


In [ ]:
# файл 004 whisper
("так не нужно самолет добраться из новосибирска до москвы я буду с двумя детьми у меня багаж две сумки по 10 килограмм найди мне билеты на 13 августа",
{"entities": [
    (13, 20, "TRANSPORT"),  # самолёт
    (31, 33, "FROM"),  # из
    (34, 46, "LOC"),  # Новосибирска
    (47, 49, "TO"),  # до
    (50, 56, "LOC"),  # Москвы
    (57, 78, "AMOUNT"), # я буду с двумя детьми
    (86, 117, "BAGG"),    # багаж две сумки по 10 килограмм
    (138, 148, "DATE"),  # 13 августа
]})

('так не нужно самолет добраться из новосибирска до москвы я буду с двумя детьми у меня багаж две сумки по 10 килограмм найди мне билеты на 13 августа',
 {'entities': [(13, 20, 'TRANSPORT'),
   (31, 33, 'FROM'),
   (34, 46, 'LOC'),
   (47, 49, 'TO'),
   (50, 56, 'LOC'),
   (57, 78, 'AMOUNT'),
   (86, 117, 'BAGG'),
   (138, 148, 'DATE')]})

In [ ]:
print_word_indices('так не нужно самолет добраться из новосибирска до москвы я буду с двумя детьми у меня багаж две сумки по 10 килограмм найди мне билеты на 13 августа')

так : 0 - 3
не : 4 - 6
нужно : 7 - 12
самолет : 13 - 20
добраться : 21 - 30
из : 31 - 33
новосибирска : 34 - 46
до : 47 - 49
москвы : 50 - 56
я : 57 - 58
буду : 59 - 63
с : 64 - 65
двумя : 66 - 71
детьми : 72 - 78
у : 79 - 80
меня : 81 - 85
багаж : 86 - 91
две : 92 - 95
сумки : 96 - 101
по : 102 - 104
10 : 105 - 107
килограмм : 108 - 117
найди : 118 - 123
мне : 124 - 127
билеты : 128 - 134
на : 135 - 137
13 : 138 - 140
августа : 141 - 148


In [ ]:
# файл 005 whisper
("поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск, подбери билеты",
{"entities": [
    (6, 8, "FROM"),  # из
    (6, 21, "LOC"),  # Архангельска
    (22, 33, "FROM"),  # в
    (24, 29, "LOC"),  # Тверь
    (30, 40, "DATE"),  # 15 октября
    (43, 53, "DATE"),  # 20 октября
    (56, 58, "FROM"),  # из
    (59, 64, "LOC"),  # Твери
    (71, 72, "TO"),  # в
    (73, 77, "LOC"),  # Омск
]})

('поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск, подбери билеты',
 {'entities': [(6, 8, 'FROM'),
   (6, 21, 'LOC'),
   (22, 33, 'FROM'),
   (24, 29, 'LOC'),
   (30, 40, 'DATE'),
   (43, 53, 'DATE'),
   (56, 58, 'FROM'),
   (59, 64, 'LOC'),
   (71, 72, 'TO'),
   (73, 77, 'LOC')]})

In [ ]:
print_word_indices('поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск, подбери билеты')

поеду : 0 - 5
из : 6 - 8
Архангельска : 9 - 21
в : 22 - 23
Тверь : 24 - 29
15 : 30 - 32
октября : 33 - 40
и : 41 - 42
20 : 43 - 45
октября : 46 - 53
я : 54 - 55
из : 56 - 58
Твери : 59 - 64
поеду : 65 - 70
в : 71 - 72
Омск, : 73 - 77
подбери : 79 - 86
билеты : 87 - 93


In [ ]:
# файл 006 whisper
("планирую из Новокузнецка выехать 30 декабря в Ростов, из Ростова нужно 8 января вернуться обратно, я буду с мужем и с ребёнком, найди билеты",
{"entities": [
    (9, 11, "FROM"),  # из
    (12, 24, "LOC"),  # Новокузнецка
    (33, 43, "DATE"),  # 30 декабря
    (44, 45, "TO"),  # в
    (46, 52, "LOC"),  # Ростов
    (54, 56, "FROM"),  # из
    (57, 64, "LOC"),  # Ростова
    (71, 79, "DATE"),  # 8 января
    (90, 97, "RETURN"),  # обратно
    (99, 126, "AMOUNT"), # я буду с мужем и с ребёнком
]})

('планирую из Новокузнецка выехать 30 декабря в Ростов, из Ростова нужно 8 января вернуться обратно, я буду с мужем и с ребёнком, найди билеты',
 {'entities': [(9, 11, 'FROM'),
   (12, 24, 'LOC'),
   (33, 43, 'DATE'),
   (44, 45, 'TO'),
   (46, 52, 'LOC'),
   (54, 56, 'FROM'),
   (57, 64, 'LOC'),
   (71, 79, 'DATE'),
   (90, 97, 'RETURN'),
   (99, 126, 'AMOUNT')]})

In [ ]:
print_word_indices('планирую из Новокузнецка выехать 30 декабря в Ростов, из Ростова нужно 8 января вернуться обратно, я буду с мужем и с ребёнком, найди билеты')

планирую : 0 - 8
из : 9 - 11
Новокузнецка : 12 - 24
выехать : 25 - 32
30 : 33 - 35
декабря : 36 - 43
в : 44 - 45
Ростов, : 46 - 52
из : 54 - 56
Ростова : 57 - 64
нужно : 65 - 70
8 : 71 - 72
января : 73 - 79
вернуться : 80 - 89
обратно, : 90 - 97
я : 99 - 100
буду : 101 - 105
с : 106 - 107
мужем : 108 - 113
и : 114 - 115
с : 116 - 117
ребёнком, : 118 - 126
найди : 128 - 133
билеты : 134 - 140


In [ ]:
# файл 007 whisper
("с 1 июля по 1 августа я поеду из Краснодара в Уфу, мне нужны билеты",
{"entities": [
    (0, 21, "GAP"),  # с 1 июля по 1 августа
    (30, 32, "FROM"),  # из
    (33, 43, "LOC"),  # Краснодара
    (44, 45, "TO"),  # в
    (46, 49, "LOC"),  # Уфу
]})

('с 1 июля по 1 августа я поеду из Краснодара в Уфу, мне нужны билеты',
 {'entities': [(0, 21, 'GAP'),
   (30, 32, 'FROM'),
   (33, 43, 'LOC'),
   (44, 45, 'TO'),
   (46, 49, 'LOC')]})

In [ ]:
print_word_indices('с 1 июля по 1 августа я поеду из Краснодара в Уфу, мне нужны билеты')

с : 0 - 1
1 : 2 - 3
июля : 4 - 8
по : 9 - 11
1 : 12 - 13
августа : 14 - 21
я : 22 - 23
поеду : 24 - 29
из : 30 - 32
Краснодара : 33 - 43
в : 44 - 45
Уфу, : 46 - 49
мне : 51 - 54
нужны : 55 - 60
билеты : 61 - 67


In [ ]:
# файл 008 whisper
("так найди-ка мне билеты из Сочи в Москву, у меня есть 5.000 и мне нужен билет на сегодня",
{"entities": [
    (24, 26, "FROM"),  # из
    (27, 31, "LOC"),  # Сочи
    (32, 33, "TO"),  # в
    (34, 40, "LOC"),  # Москву
    (54, 59, "BUDGET"),   # 5.000
    (78, 88, "DATE"),  # на сегодня
]})

('так найди-ка мне билеты из Сочи в Москву, у меня есть 5.000 и мне нужен билет на сегодня',
 {'entities': [(24, 26, 'FROM'),
   (27, 31, 'LOC'),
   (32, 33, 'TO'),
   (34, 40, 'LOC'),
   (54, 59, 'BUDGET'),
   (78, 88, 'DATE')]})

In [ ]:
print_word_indices('так найди-ка мне билеты из Сочи в Москву, у меня есть 5.000 и мне нужен билет на сегодня')

так : 0 - 3
найди-ка : 4 - 12
мне : 13 - 16
билеты : 17 - 23
из : 24 - 26
Сочи : 27 - 31
в : 32 - 33
Москву, : 34 - 40
у : 42 - 43
меня : 44 - 48
есть : 49 - 53
5.000 : 54 - 59
и : 60 - 61
мне : 62 - 65
нужен : 66 - 71
билет : 72 - 77
на : 78 - 80
сегодня : 81 - 88


In [ ]:
# файл 009 whisper
("7 января хочу на самолете долететь из Омска в Тобольск, подбери билет, я буду одна, у меня с собой ручная кладь",
{"entities": [
    (0, 8, "DATE"),  # 7 января
    (14, 25, "TRANSPORT"),  # на самолете
    (35, 37, "FROM"),  # из
    (38, 43, "LOC"),  # Омска
    (44, 45, "TO"),  # в
    (46, 54, "LOC"),  # Тобольск
    (71, 82, "AMOUNT"), # я буду одна
    (99, 111, "BAGG"),    # ручная кладь
]})

('7 января хочу на самолете долететь из Омска в Тобольск, подбери билет, я буду одна, у меня с собой ручная кладь',
 {'entities': [(0, 8, 'DATE'),
   (14, 25, 'TRANSPORT'),
   (35, 37, 'FROM'),
   (38, 43, 'LOC'),
   (44, 45, 'TO'),
   (46, 54, 'LOC'),
   (71, 82, 'AMOUNT'),
   (99, 111, 'BAGG')]})

In [ ]:
print_word_indices('7 января хочу на самолете долететь из Омска в Тобольск, подбери билет, я буду одна, у меня с собой ручная кладь')

7 : 0 - 1
января : 2 - 8
хочу : 9 - 13
на : 14 - 16
самолете : 17 - 25
долететь : 26 - 34
из : 35 - 37
Омска : 38 - 43
в : 44 - 45
Тобольск, : 46 - 54
подбери : 56 - 63
билет, : 64 - 69
я : 71 - 72
буду : 73 - 77
одна, : 78 - 82
у : 84 - 85
меня : 86 - 90
с : 91 - 92
собой : 93 - 98
ручная : 99 - 105
кладь : 106 - 111


In [ ]:
# файл 010 whisper
("с 14 по 20 февраля я поеду из Твери в Москву и потом обратно, я буду одна, подбери самый дешёвый билет",
{"entities": [
    (0, 18, "GAP"),  # с 14 по 20 февраля
    (27, 29, "FROM"),  # из
    (30, 35, "LOC"),  # Твери
    (36, 37, "TO"),  # в
    (38, 44, "LOC"),  # Москву
    (53, 60, "RETURN"), # обратно
    (62, 73, "AMOUNT"), # я буду одна
    (75, 102, "LEVEL"), # подбери самый дешёвый билет
]})

('с 14 по 20 февраля я поеду из Твери в Москву и потом обратно, я буду одна, подбери самый дешёвый билет',
 {'entities': [(0, 18, 'GAP'),
   (27, 29, 'FROM'),
   (30, 35, 'LOC'),
   (36, 37, 'TO'),
   (38, 44, 'LOC'),
   (53, 60, 'RETURN'),
   (62, 73, 'AMOUNT'),
   (75, 102, 'LEVEL')]})

In [ ]:
print_word_indices('с 14 по 20 февраля я поеду из Твери в Москву и потом обратно, я буду одна, подбери самый дешёвый билет')

с : 0 - 1
14 : 2 - 4
по : 5 - 7
20 : 8 - 10
февраля : 11 - 18
я : 19 - 20
поеду : 21 - 26
из : 27 - 29
Твери : 30 - 35
в : 36 - 37
Москву : 38 - 44
и : 45 - 46
потом : 47 - 52
обратно, : 53 - 60
я : 62 - 63
буду : 64 - 68
одна, : 69 - 73
подбери : 75 - 82
самый : 83 - 88
дешёвый : 89 - 96
билет : 97 - 102


In [ ]:
# файл 011 whisper
("хочу из Орла поехать в Пензу 20 марта и потом из Пензы мне нужно в Екатеринбург. это, наверное, число 25 марта. я буду одна, но у меня с собой будет чемодан, примерно 20 килограмм",
{"entities": [
    (5, 7, "FROM"),  # из
    (8, 12, "LOC"),  # Орла
    (21, 22, "TO"),  # в
    (30, 35, "LOC"),  # Пензу
    (29, 37, "DATE"),  # 20 марта
    (46, 48, "FROM"),  # из
    (49, 54, "LOC"),  # Пензы
    (65, 66, "TO"),  # в
    (67, 79, "LOC"),  # Екатеринбург
    (102, 110, "DATE"),  # 25 марта
    (112, 123, "AMOUNT"), # я буду одна
    (149, 179, "BAGG"),    # чемодан, примерно 20 килограмм
]})

('хочу из Орла поехать в Пензу 20 марта и потом из Пензы мне нужно в Екатеринбург. это, наверное, число 25 марта. я буду одна, но у меня с собой будет чемодан, примерно 20 килограмм',
 {'entities': [(5, 7, 'FROM'),
   (8, 12, 'LOC'),
   (21, 22, 'TO'),
   (30, 35, 'LOC'),
   (29, 37, 'DATE'),
   (46, 48, 'FROM'),
   (49, 54, 'LOC'),
   (65, 66, 'TO'),
   (67, 79, 'LOC'),
   (102, 110, 'DATE'),
   (112, 123, 'AMOUNT'),
   (149, 179, 'BAGG')]})

In [ ]:
print_word_indices('хочу из Орла поехать в Пензу 20 марта и потом из Пензы мне нужно в Екатеринбург. это, наверное, число 25 марта. я буду одна, но у меня с собой будет чемодан, примерно 20 килограмм')

хочу : 0 - 4
из : 5 - 7
Орла : 8 - 12
поехать : 13 - 20
в : 21 - 22
Пензу : 23 - 28
20 : 29 - 31
марта : 32 - 37
и : 38 - 39
потом : 40 - 45
из : 46 - 48
Пензы : 49 - 54
мне : 55 - 58
нужно : 59 - 64
в : 65 - 66
Екатеринбург. : 67 - 79
это, : 81 - 84
наверное, : 86 - 94
число : 96 - 101
25 : 102 - 104
марта. : 105 - 110
я : 112 - 113
буду : 114 - 118
одна, : 119 - 123
но : 125 - 127
у : 128 - 129
меня : 130 - 134
с : 135 - 136
собой : 137 - 142
будет : 143 - 148
чемодан, : 149 - 156
примерно : 158 - 166
20 : 167 - 169
килограмм : 170 - 179


In [ ]:
# файл 012 whisper
("1 апреля я вылетаю из Москвы в Волгоград и из Волгограда мне нужно будет на поезде доехать до Нижневартовска, подбери билет и я буду с мужем.",
{"entities": [
    (0, 8, "DATE"),  # 1 апреля
    (11, 18, "TRANSPORT"),  # вылетаю
    (19, 21, "FROM"),  # из
    (22, 28, "LOC"),  # Москвы
    (29, 30, "TO"),  # в
    (31, 40, "LOC"),  # Волгоград
    (43, 45, "FROM"),  # из
    (46, 56, "LOC"),  # Волгограда
    (73, 82, "TRANSPORT"),  # на поезде
    (91, 93, "TO"),  # до
    (94, 108, "LOC"),  # Нижневартовска
    (126, 140, "AMOUNT"), # я буду с мужем
]})

('1 апреля я вылетаю из Москвы в Волгоград и из Волгограда мне нужно будет на поезде доехать до Нижневартовска, подбери билет и я буду с мужем.',
 {'entities': [(0, 8, 'DATE'),
   (11, 18, 'TRANSPORT'),
   (19, 21, 'FROM'),
   (22, 28, 'LOC'),
   (29, 30, 'TO'),
   (31, 40, 'LOC'),
   (43, 45, 'FROM'),
   (46, 56, 'LOC'),
   (73, 82, 'TRANSPORT'),
   (91, 93, 'TO'),
   (94, 108, 'LOC'),
   (126, 140, 'AMOUNT')]})

In [ ]:
print_word_indices('1 апреля я вылетаю из Москвы в Волгоград и из Волгограда мне нужно будет на поезде доехать до Нижневартовска, подбери билет и я буду с мужем.')

1 : 0 - 1
апреля : 2 - 8
я : 9 - 10
вылетаю : 11 - 18
из : 19 - 21
Москвы : 22 - 28
в : 29 - 30
Волгоград : 31 - 40
и : 41 - 42
из : 43 - 45
Волгограда : 46 - 56
мне : 57 - 60
нужно : 61 - 66
будет : 67 - 72
на : 73 - 75
поезде : 76 - 82
доехать : 83 - 90
до : 91 - 93
Нижневартовска, : 94 - 108
подбери : 110 - 117
билет : 118 - 123
и : 124 - 125
я : 126 - 127
буду : 128 - 132
с : 133 - 134
мужем. : 135 - 140


In [ ]:
# файл 013 whisper
("так с 1 по 9 мая я еду отдыхать в Сочи из Улан-удэ, я поеду с мужем и двумя детьми, подбери самые быстрые билеты.",
{"entities": [
    (4, 16, "GAP"),  # с 1 по 9 мая
    (32, 33, "TO"),  # в
    (34, 38, "LOC"),  # Сочи
    (39, 41, "FROM"),  # из
    (42, 50, "LOC"),  # Улан-удэ
    (52, 82, "AMOUNT"), # я поеду с мужем и двумя детьми
    (84, 112, "LEVEL"), # подбери самые быстрые билеты
]})

('так с 1 по 9 мая я еду отдыхать в Сочи из Улан-удэ, я поеду с мужем и двумя детьми, подбери самые быстрые билеты.',
 {'entities': [(4, 16, 'GAP'),
   (32, 33, 'TO'),
   (34, 38, 'LOC'),
   (39, 41, 'FROM'),
   (42, 50, 'LOC'),
   (52, 82, 'AMOUNT'),
   (84, 112, 'LEVEL')]})

In [ ]:
print_word_indices('так с 1 по 9 мая я еду отдыхать в Сочи из Улан-удэ, я поеду с мужем и двумя детьми, подбери самые быстрые билеты.')

так : 0 - 3
с : 4 - 5
1 : 6 - 7
по : 8 - 10
9 : 11 - 12
мая : 13 - 16
я : 17 - 18
еду : 19 - 22
отдыхать : 23 - 31
в : 32 - 33
Сочи : 34 - 38
из : 39 - 41
Улан-удэ, : 42 - 50
я : 52 - 53
поеду : 54 - 59
с : 60 - 61
мужем : 62 - 67
и : 68 - 69
двумя : 70 - 75
детьми, : 76 - 82
подбери : 84 - 91
самые : 92 - 97
быстрые : 98 - 105
билеты. : 106 - 112


In [ ]:
# файл 014 whisper
("с 3 по 17 июня я поеду из Нижневартовска в Астрахань, я буду с коллегой, нам нужно два билета туда-обратно.",
{"entities": [
    (0, 14, "GAP"),  # с 3 по 17 июня
    (23, 25, "FROM"),  # из
    (26, 40, "LOC"),  # Нижневартовска
    (41, 42, "TO"),  # в
    (43, 52, "LOC"),  # Астрахань
    (54, 71, "AMOUNT"), # я буду с коллегой
    (83, 93, "AMOUNT"), # два билета
    (94, 106, "RETURN"), # туда-обратно
]})

('с 3 по 17 июня я поеду из Нижневартовска в Астрахань, я буду с коллегой, нам нужно два билета туда-обратно.',
 {'entities': [(0, 14, 'GAP'),
   (23, 25, 'FROM'),
   (26, 40, 'LOC'),
   (41, 42, 'TO'),
   (43, 52, 'LOC'),
   (54, 71, 'AMOUNT'),
   (83, 93, 'AMOUNT'),
   (94, 106, 'RETURN')]})

In [ ]:
print_word_indices('с 3 по 17 июня я поеду из Нижневартовска в Астрахань, я буду с коллегой, нам нужно два билета туда-обратно.')

с : 0 - 1
3 : 2 - 3
по : 4 - 6
17 : 7 - 9
июня : 10 - 14
я : 15 - 16
поеду : 17 - 22
из : 23 - 25
Нижневартовска : 26 - 40
в : 41 - 42
Астрахань, : 43 - 52
я : 54 - 55
буду : 56 - 60
с : 61 - 62
коллегой, : 63 - 71
нам : 73 - 76
нужно : 77 - 82
два : 83 - 86
билета : 87 - 93
туда-обратно. : 94 - 106


In [ ]:
# файл 015 whisper
("так мне нужно из Краснодара на автобусе добраться до Сочи, посмотри пожалуйста билеты, у меня будет ручная кладь и ребёнок.",
{"entities": [
    (14, 16, "FROM"),  # из
    (17, 27, "LOC"),  # Краснодара
    (31, 39, "TRANSPORT"),  # автобусе
    (50, 52, "TO"),  # до
    (53, 57, "LOC"),  # Сочи
    (100, 112, "BAGG"),    # ручная кладь
    (115, 122, "CHILD"),  # ребёнок
]})

('так мне нужно из Краснодара на автобусе добраться до Сочи, посмотри пожалуйста билеты, у меня будет ручная кладь и ребёнок.',
 {'entities': [(14, 16, 'FROM'),
   (17, 27, 'LOC'),
   (31, 39, 'TRANSPORT'),
   (50, 52, 'TO'),
   (53, 57, 'LOC'),
   (100, 112, 'BAGG'),
   (115, 122, 'CHILD')]})

In [ ]:
print_word_indices('так мне нужно из Краснодара на автобусе добраться до Сочи, посмотри пожалуйста билеты, у меня будет ручная кладь и ребёнок.')

так : 0 - 3
мне : 4 - 7
нужно : 8 - 13
из : 14 - 16
Краснодара : 17 - 27
на : 28 - 30
автобусе : 31 - 39
добраться : 40 - 49
до : 50 - 52
Сочи, : 53 - 57
посмотри : 59 - 67
пожалуйста : 68 - 78
билеты, : 79 - 85
у : 87 - 88
меня : 89 - 93
будет : 94 - 99
ручная : 100 - 106
кладь : 107 - 112
и : 113 - 114
ребёнок. : 115 - 122


In [ ]:
# файл 016 whisper
("получается 3 июля по 13 июля я поеду с ребёнком в Дагестан из Москвы, багаж будет 30 кг.",
{"entities": [
    (11, 28, "GAP"),  # 3 июля по 13 июля
    (29, 47, "CHILD"),  # с ребёнком
    (48, 49, "TO"),  # в
    (50, 58, "LOC"),  # Дагестан
    (59, 61, "FROM"),  # из
    (62, 68, "LOC"),  # Москвы
    (70, 87, "BAGG"),    # багаж будет 30 кг
]})

('получается 3 июля по 13 июля я поеду с ребёнком в Дагестан из Москвы, багаж будет 30 кг.',
 {'entities': [(11, 28, 'GAP'),
   (29, 47, 'CHILD'),
   (48, 49, 'TO'),
   (50, 58, 'LOC'),
   (59, 61, 'FROM'),
   (62, 68, 'LOC'),
   (70, 87, 'BAGG')]})

In [ ]:
print_word_indices('получается 3 июля по 13 июля я поеду с ребёнком в Дагестан из Москвы, багаж будет 30 кг.')

получается : 0 - 10
3 : 11 - 12
июля : 13 - 17
по : 18 - 20
13 : 21 - 23
июля : 24 - 28
я : 29 - 30
поеду : 31 - 36
с : 37 - 38
ребёнком : 39 - 47
в : 48 - 49
Дагестан : 50 - 58
из : 59 - 61
Москвы, : 62 - 68
багаж : 70 - 75
будет : 76 - 81
30 : 82 - 84
кг. : 85 - 87


In [ ]:
# файл 017 whisper
("2 августа я поеду из Астрахани в Махачкалу, подбери пожалуйста билет, у меня на поездку 7.000 руб.",
{"entities": [
    (0, 9, "DATE"),  # 2 августа
    (18, 20, "FROM"),  # из
    (21, 30, "LOC"),  # Астрахани
    (31, 32, "TO"),  # в
    (33, 42, "LOC"),  # Махачкалу
    (54, 59, "BUDGET"),  # 7.000 руб.
]})

('2 августа я поеду из Астрахани в Махачкалу, подбери пожалуйста билет, у меня на поездку 7.000 руб.',
 {'entities': [(0, 9, 'DATE'),
   (18, 20, 'FROM'),
   (21, 30, 'LOC'),
   (31, 32, 'TO'),
   (33, 42, 'LOC'),
   (54, 59, 'BUDGET')]})

In [ ]:
print_word_indices('2 августа я поеду из Астрахани в Махачкалу, подбери пожалуйста билет, у меня на поездку 7.000 руб.')

2 : 0 - 1
августа : 2 - 9
я : 10 - 11
поеду : 12 - 17
из : 18 - 20
Астрахани : 21 - 30
в : 31 - 32
Махачкалу, : 33 - 42
подбери : 44 - 51
пожалуйста : 52 - 62
билет, : 63 - 68
у : 70 - 71
меня : 72 - 76
на : 77 - 79
поездку : 80 - 87
7.000 : 88 - 93
руб. : 94 - 97


In [ ]:
# файл 018 whisper
("так я из Нефтекамска хочу поехать в Улан-удэ, это будет значит с 8 сентября по 19 число, подбери билет туда-обратно и у меня поедет муж, а ему нужно обязательно на поезде нижняя полочка.",
{"entities": [
    (6, 8, "FROM"),  # из
    (9, 20, "LOC"),  # Нефтекамска
    (34, 35, "TO"),  # в
    (36, 44, "LOC"),  # Улан-удэ
    (63, 81, "GAP"),  # с 8 сентября по 19
    (103, 115, "RETURN"),  # туда-обратно
    (118, 135, "AMOUNT"),  # у меня поедет муж
    (164, 170, "TRANSPORT"),  # поезде
    (171, 185, "COMFORT"),  # нижняя полочка

]})

('так я из Нефтекамска хочу поехать в Улан-удэ, это будет значит с 8 сентября по 19 число, подбери билет туда-обратно и у меня поедет муж, а ему нужно обязательно на поезде нижняя полочка.',
 {'entities': [(6, 8, 'FROM'),
   (9, 20, 'LOC'),
   (34, 35, 'TO'),
   (36, 44, 'LOC'),
   (63, 81, 'GAP'),
   (103, 115, 'RETURN'),
   (118, 135, 'AMOUNT'),
   (164, 170, 'TRANSPORT'),
   (171, 185, 'COMFORT')]})

In [ ]:
print_word_indices('так я из Нефтекамска хочу поехать в Улан-удэ, это будет значит с 8 сентября по 19 число, подбери билет туда-обратно и у меня поедет муж, а ему нужно обязательно на поезде нижняя полочка.')

так : 0 - 3
я : 4 - 5
из : 6 - 8
Нефтекамска : 9 - 20
хочу : 21 - 25
поехать : 26 - 33
в : 34 - 35
Улан-удэ, : 36 - 44
это : 46 - 49
будет : 50 - 55
значит : 56 - 62
с : 63 - 64
8 : 65 - 66
сентября : 67 - 75
по : 76 - 78
19 : 79 - 81
число, : 82 - 87
подбери : 89 - 96
билет : 97 - 102
туда-обратно : 103 - 115
и : 116 - 117
у : 118 - 119
меня : 120 - 124
поедет : 125 - 131
муж, : 132 - 135
а : 137 - 138
ему : 139 - 142
нужно : 143 - 148
обязательно : 149 - 160
на : 161 - 163
поезде : 164 - 170
нижняя : 171 - 177
полочка. : 178 - 185


In [ ]:
# файл 019 whisper
("так 18 октября я поеду в командировку на Шарам, вот и я там буду неделю, и мне нужен билет, я поеду из Омска, посмотри как быстрее всего добраться.",
{"entities": [
    (4, 14, "DATE"),  # 18 октября
    (38, 40, "TO"),  # на
    (41, 46, "LOC"),  # Шарам
    (100, 102, "FROM"),  # из
    (103, 108, "LOC"),  # Омска
]})

('так 18 октября я поеду в командировку на Шарам, вот и я там буду неделю, и мне нужен билет, я поеду из Омска, посмотри как быстрее всего добраться.',
 {'entities': [(4, 14, 'DATE'),
   (38, 40, 'TO'),
   (41, 46, 'LOC'),
   (100, 102, 'FROM'),
   (103, 108, 'LOC')]})

In [ ]:
print_word_indices('так 18 октября я поеду в командировку на Шарам, вот и я там буду неделю, и мне нужен билет, я поеду из Омска, посмотри как быстрее всего добраться.')

так : 0 - 3
18 : 4 - 6
октября : 7 - 14
я : 15 - 16
поеду : 17 - 22
в : 23 - 24
командировку : 25 - 37
на : 38 - 40
Шарам, : 41 - 46
вот : 48 - 51
и : 52 - 53
я : 54 - 55
там : 56 - 59
буду : 60 - 64
неделю, : 65 - 71
и : 73 - 74
мне : 75 - 78
нужен : 79 - 84
билет, : 85 - 90
я : 92 - 93
поеду : 94 - 99
из : 100 - 102
Омска, : 103 - 108
посмотри : 110 - 118
как : 119 - 122
быстрее : 123 - 130
всего : 131 - 136
добраться. : 137 - 146


In [ ]:
# файл 020 whisper
("15 ноября я поеду в Нальчик. Я поеду из Томска. Найди, пожалуйста, мне билеты до 10 тысяч рублей.",
{"entities": [
    (0, 9, "DATE"),  # 15 ноября
    (18, 19, "TO"),  # в
    (20, 27, "LOC"),  # Нальчик
    (37, 39, "FROM"),  # из
    (40, 46, "LOC"),  # Томска
    (81, 96, "BUDGET"),  # 10 тысяч рублей.
]})

('15 ноября я поеду в Нальчик. Я поеду из Томска. Найди, пожалуйста, мне билеты до 10 тысяч рублей.',
 {'entities': [(0, 9, 'DATE'),
   (18, 19, 'TO'),
   (20, 27, 'LOC'),
   (37, 39, 'FROM'),
   (40, 46, 'LOC'),
   (81, 96, 'BUDGET')]})

In [ ]:
print_word_indices('15 ноября я поеду в Нальчик. Я поеду из Томска. Найди, пожалуйста, мне билеты до 10 тысяч рублей.')

15 : 0 - 2
ноября : 3 - 9
я : 10 - 11
поеду : 12 - 17
в : 18 - 19
Нальчик. : 20 - 27
Я : 29 - 30
поеду : 31 - 36
из : 37 - 39
Томска. : 40 - 46
Найди, : 48 - 53
пожалуйста, : 55 - 65
мне : 67 - 70
билеты : 71 - 77
до : 78 - 80
10 : 81 - 83
тысяч : 84 - 89
рублей. : 90 - 96


In [ ]:
# файл 021 whisper
("6 декабря 6 декабря точнее А так значит по 10 декабря Я еду в Москву из Норильска Мне нужен самый быстрый билет Найди пожалуйста.",
{"entities": [
    (0, 9, "DATE"),  # 6 декабря
    (10, 53, "GAP"),  # 6 декабря точнее А так значит по 10 декабря
    (60, 61, "TO"),  # в
    (62, 68, "LOC"),  # Москву
    (69, 71, "FROM"),  # из
    (72, 81, "LOC"),  # Норильска
    (92, 111, "COMFORT"),  # самый быстрый билет
]})

('6 декабря 6 декабря точнее А так значит по 10 декабря Я еду в Москву из Норильска Мне нужен самый быстрый билет Найди пожалуйста.',
 {'entities': [(0, 9, 'DATE'),
   (10, 53, 'GAP'),
   (60, 61, 'TO'),
   (62, 68, 'LOC'),
   (69, 71, 'FROM'),
   (72, 81, 'LOC'),
   (92, 111, 'COMFORT')]})

In [ ]:
print_word_indices('6 декабря 6 декабря точнее А так значит по 10 декабря Я еду в Москву из Норильска Мне нужен самый быстрый билет Найди пожалуйста.')

6 : 0 - 1
декабря : 2 - 9
6 : 10 - 11
декабря : 12 - 19
точнее : 20 - 26
А : 27 - 28
так : 29 - 32
значит : 33 - 39
по : 40 - 42
10 : 43 - 45
декабря : 46 - 53
Я : 54 - 55
еду : 56 - 59
в : 60 - 61
Москву : 62 - 68
из : 69 - 71
Норильска : 72 - 81
Мне : 82 - 85
нужен : 86 - 91
самый : 92 - 97
быстрый : 98 - 105
билет : 106 - 111
Найди : 112 - 117
пожалуйста. : 118 - 128


In [ ]:
# файл 022 whisper
("Покажи сегодня ближайшие рейсы до Екатеринбурга. Можно с пересадкой в Новосибирск, транзит. Только, пожалуйста, ближайшие покажи мне сегодня.",
{"entities": [
    (7, 14, "DATE"),  # сегодня
    (31, 33, "TO"),  # до
    (34, 47, "LOC"),  # Екатеринбурга
    (68, 69, "TO"),  # в
    (20, 27, "LOC"),  # Новосибирск
    (55, 67, "TRANSIT"),  # транзит
    (133, 140, "DATE"),  # сегодня.
]})

('Покажи сегодня ближайшие рейсы до Екатеринбурга. Можно с пересадкой в Новосибирск, транзит. Только, пожалуйста, ближайшие покажи мне сегодня.',
 {'entities': [(7, 14, 'DATE'),
   (31, 33, 'TO'),
   (34, 47, 'LOC'),
   (68, 69, 'TO'),
   (20, 27, 'LOC'),
   (55, 67, 'TRANSIT'),
   (133, 140, 'DATE')]})

In [ ]:
print_word_indices('Покажи сегодня ближайшие рейсы до Екатеринбурга. Можно с пересадкой в Новосибирск, транзит. Только, пожалуйста, ближайшие покажи мне сегодня.')

Покажи : 0 - 6
сегодня : 7 - 14
ближайшие : 15 - 24
рейсы : 25 - 30
до : 31 - 33
Екатеринбурга. : 34 - 47
Можно : 49 - 54
с : 55 - 56
пересадкой : 57 - 67
в : 68 - 69
Новосибирск, : 70 - 81
транзит. : 83 - 90
Только, : 92 - 98
пожалуйста, : 100 - 110
ближайшие : 112 - 121
покажи : 122 - 128
мне : 129 - 132
сегодня. : 133 - 140


In [ ]:
# файл 023 whisper
("Поезда отменили. Покажи ближайший самолет до Кавменвод из Краснодара на завтра, 5 число утра.",
{"entities": [
    (0, 6, "TRANSPORT"),  # Поезда
    (34, 41, "TRANSPORT"),  # самолет
    (42, 44, "TO"),  # до
    (45, 54, "LOC"),  # Кавменвод
    (55, 57, "FROM"),  # из
    (58, 68, "LOC"),  # Краснодара
    (72, 78, "DATE"),  # завтра
    (80, 92, "DATE"),  # 5 число утра.
]})

('Поезда отменили. Покажи ближайший самолет до Кавменвод из Краснодара на завтра, 5 число утра.',
 {'entities': [(0, 6, 'TRANSPORT'),
   (34, 41, 'TRANSPORT'),
   (42, 44, 'TO'),
   (45, 54, 'LOC'),
   (55, 57, 'FROM'),
   (58, 68, 'LOC'),
   (72, 78, 'DATE'),
   (80, 92, 'DATE')]})

In [ ]:
print_word_indices('Поезда отменили. Покажи ближайший самолет до Кавменвод из Краснодара на завтра, 5 число утра.')

Поезда : 0 - 6
отменили. : 7 - 15
Покажи : 17 - 23
ближайший : 24 - 33
самолет : 34 - 41
до : 42 - 44
Кавменвод : 45 - 54
из : 55 - 57
Краснодара : 58 - 68
на : 69 - 71
завтра, : 72 - 78
5 : 80 - 81
число : 82 - 87
утра. : 88 - 92


In [ ]:
# файл 024 whisper
("Посмотри пожалуйста есть ли электрички от Тулы до Москвы на эти выходные 5-6 октября. Любое время все что есть найди покажи.",
{"entities": [
    (28, 38, "TRANSPORT"),  # электрички
    (39, 41, "FROM"),  # от
    (42, 46, "LOC"),  # Тулы
    (47, 49, "TO"),  # до
    (50, 56, "LOC"),  # Москвы
    (73, 84, "DATE"),  # 5-6 октября
]})

('Посмотри пожалуйста есть ли электрички от Тулы до Москвы на эти выходные 5-6 октября. Любое время все что есть найди покажи.',
 {'entities': [(28, 38, 'TRANSPORT'),
   (39, 41, 'FROM'),
   (42, 46, 'LOC'),
   (47, 49, 'TO'),
   (50, 56, 'LOC'),
   (73, 84, 'DATE')]})

In [ ]:
print_word_indices('Посмотри пожалуйста есть ли электрички от Тулы до Москвы на эти выходные 5-6 октября. Любое время все что есть найди покажи.')

Посмотри : 0 - 8
пожалуйста : 9 - 19
есть : 20 - 24
ли : 25 - 27
электрички : 28 - 38
от : 39 - 41
Тулы : 42 - 46
до : 47 - 49
Москвы : 50 - 56
на : 57 - 59
эти : 60 - 63
выходные : 64 - 72
5-6 : 73 - 76
октября. : 77 - 84
Любое : 86 - 91
время : 92 - 97
все : 98 - 101
что : 102 - 105
есть : 106 - 110
найди : 111 - 116
покажи. : 117 - 123


In [ ]:
# файл 025 whisper
("Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть, с прибытием не позже шести утра следующего дня.",
{"entities": [
    (14, 19, "TRANSPORT"),  # поезд
    (41, 50, "DATE"),  # 20 ноября
    (51, 53, "FROM"),  # из
    (54, 75, "LOC"),  # Комсомольска на Амуре
    (76, 78, "TO"),  # до
    (79, 89, "LOC"),  # Хабаровска
    (116, 162, "DATE"),  # с прибытием не позже шести утра следующего дня.
]})

('Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть, с прибытием не позже шести утра следующего дня.',
 {'entities': [(14, 19, 'TRANSPORT'),
   (41, 50, 'DATE'),
   (51, 53, 'FROM'),
   (54, 75, 'LOC'),
   (76, 78, 'TO'),
   (79, 89, 'LOC'),
   (116, 162, 'DATE')]})

In [ ]:
print_word_indices('Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть, с прибытием не позже шести утра следующего дня.')

Срочно : 0 - 6
покажи : 7 - 13
поезд : 14 - 19
ближайший : 20 - 29
на : 30 - 32
сегодня : 33 - 40
20 : 41 - 43
ноября : 44 - 50
из : 51 - 53
Комсомольска : 54 - 66
на : 67 - 69
Амуре : 70 - 75
до : 76 - 78
Хабаровска, : 79 - 89
ближайший, : 91 - 100
который : 102 - 109
есть, : 110 - 114
с : 116 - 117
прибытием : 118 - 127
не : 128 - 130
позже : 131 - 136
шести : 137 - 142
утра : 143 - 147
следующего : 148 - 158
дня. : 159 - 162


In [ ]:
# файл 026 whisper
("покажи мне, вообще, все варианты полетов из Кавминвод до Вилюйска, неважно пересадки, не важно время, главное чтобы вылет был послезавтра 7 июля.",
{"entities": [
    (33, 40, "TRANSPORT"),  # полетов
    (41, 43, "FROM"),  # из
    (44, 53, "LOC"),  # Кавминвод
    (54, 56, "TO"),  # до
    (57, 65, "LOC"),  # Вилюйска
    (138, 144, "DATE"),  # 7 июля.
]})

('покажи мне, вообще, все варианты полетов из Кавминвод до Вилюйска, неважно пересадки, не важно время, главное чтобы вылет был послезавтра 7 июля.',
 {'entities': [(33, 40, 'TRANSPORT'),
   (41, 43, 'FROM'),
   (44, 53, 'LOC'),
   (54, 56, 'TO'),
   (57, 65, 'LOC'),
   (138, 144, 'DATE')]})

In [ ]:
print_word_indices('покажи мне, вообще, все варианты полетов из Кавминвод до Вилюйска, неважно пересадки, не важно время, главное чтобы вылет был послезавтра 7 июля.')

покажи : 0 - 6
мне, : 7 - 10
вообще, : 12 - 18
все : 20 - 23
варианты : 24 - 32
полетов : 33 - 40
из : 41 - 43
Кавминвод : 44 - 53
до : 54 - 56
Вилюйска, : 57 - 65
неважно : 67 - 74
пересадки, : 75 - 84
не : 86 - 88
важно : 89 - 94
время, : 95 - 100
главное : 102 - 109
чтобы : 110 - 115
вылет : 116 - 121
был : 122 - 125
послезавтра : 126 - 137
7 : 138 - 139
июля. : 140 - 144


In [ ]:
# файл 027 whisper
("покажи мне на декабрь, к примеру, 5 декабря, самолеты из Москвы до Петропавловска-Камчатский интересные варианты, которые будут прямые, без пересадки, обратно пока не показывай.",
{"entities": [
    (11, 21, "DATE"),  # на декабрь
    (34, 43, "DATE"),  # 5 декабря
    (45, 53, "TRANSPORT"),  # самолеты
    (54, 56, "FROM"),  # из
    (57, 63, "LOC"),  # Москвы
    (64, 66, "TO"),  # до
    (67, 92, "LOC"),  # Петропавловска-Камчатский
    (128, 149, "TRANZIT"),  # прямые, без пересадки
    (151, 176, "RETURN"),  # обратно пока не показывай.
]})

('покажи мне на декабрь, к примеру, 5 декабря, самолеты из Москвы до Петропавловска-Камчатский интересные варианты, которые будут прямые, без пересадки, обратно пока не показывай.',
 {'entities': [(11, 21, 'DATE'),
   (34, 43, 'DATE'),
   (45, 53, 'TRANSPORT'),
   (54, 56, 'FROM'),
   (57, 63, 'LOC'),
   (64, 66, 'TO'),
   (67, 92, 'LOC'),
   (128, 149, 'TRANZIT'),
   (151, 176, 'RETURN')]})

In [ ]:
print_word_indices('покажи мне на декабрь, к примеру, 5 декабря, самолеты из Москвы до Петропавловска-Камчатский интересные варианты, которые будут прямые, без пересадки, обратно пока не показывай.')

покажи : 0 - 6
мне : 7 - 10
на : 11 - 13
декабрь, : 14 - 21
к : 23 - 24
примеру, : 25 - 32
5 : 34 - 35
декабря, : 36 - 43
самолеты : 45 - 53
из : 54 - 56
Москвы : 57 - 63
до : 64 - 66
Петропавловска-Камчатский : 67 - 92
интересные : 93 - 103
варианты, : 104 - 112
которые : 114 - 121
будут : 122 - 127
прямые, : 128 - 134
без : 136 - 139
пересадки, : 140 - 149
обратно : 151 - 158
пока : 159 - 163
не : 164 - 166
показывай. : 167 - 176


In [ ]:
# файл 028 whisper
("срочно, на завтра, поезд утренний, Тула  Казань вообще все варианты, неважно какая цена, главное чтобы утром.",
{"entities": [
    (11, 21, "DATE"),  # завтра
    (45, 53, "TRANSPORT"),  # поезд
    (11, 21, "DATE"),  # утренний
    (57, 63, "LOC"),  # Тула
    (57, 63, "LOC"),  # Казань
    (81, 96, "BUDGET"),  # неважно какая цена
    (34, 43, "COMFORT"),  # утром
]})

('срочно, на завтра, поезд утренний, Тула  Казань вообще все варианты, неважно какая цена, главное чтобы утром.',
 {'entities': [(11, 21, 'DATE'),
   (45, 53, 'TRANSPORT'),
   (11, 21, 'DATE'),
   (57, 63, 'LOC'),
   (57, 63, 'LOC'),
   (81, 96, 'BUDGET'),
   (34, 43, 'COMFORT')]})

In [ ]:
print_word_indices('срочно, на завтра, поезд утренний, Тула - Казань вообще все варианты, неважно какая цена, главное чтобы утром.')

срочно, : 0 - 6
на : 8 - 10
завтра, : 11 - 17
поезд : 19 - 24
утренний, : 25 - 33
Тула : 35 - 39
- : 40 - 40
Казань : 42 - 48
вообще : 49 - 55
все : 56 - 59
варианты, : 60 - 68
неважно : 70 - 77
какая : 78 - 83
цена, : 84 - 88
главное : 90 - 97
чтобы : 98 - 103
утром. : 104 - 109


In [ ]:
# файл 029 whisper
("построй пожалуйста маршрут, самолёты на 1 мая из Ярославля в Иркутск, потом сутки ночуем и третьего числа вылетаем из Иркутска в Петропавловск-камчатский, и с Петропавловска-камчатский, где-то через недельку, покажи, пожалуйста, маршруты с 15, прямой рейс до Москвы.",
{"entities": [
    (45, 53, "TRANSPORT"),  # самолёты
    (11, 21, "DATE"),  # 1 мая
    (57, 63, "FROM"),  # из
    (57, 63, "LOC"),  # Ярославля
    (57, 63, "TO"),  # в
    (57, 63, "LOC"),  # Иркутск
    (11, 21, "DATE"),  # третьего числа
    (45, 53, "TRANSPORT"),  # вылетаем
    (57, 63, "FROM"),  # из
    (57, 63, "LOC"),  # Иркутска
    (57, 63, "TO"),  # в
    (57, 63, "LOC"),  # Петропавловск-камчатский
    (57, 63, "FROM"),  # из
    (57, 63, "LOC"),  # Петропавловск-камчатский
    (11, 21, "DATE"),  # через недельку              маршруты с 15, прямой рейс до Москвы.
    (11, 21, "DATE"),  # с 15
    (45, 53, "TRANSPORT"),  # прямой рейс
    (57, 63, "TO"),  # до
    (57, 63, "LOC"),  # Москвы.
]})

('построй пожалуйста маршрут, самолёты на 1 мая из Ярославля в Иркутск, потом сутки ночуем и третьего числа вылетаем из Иркутска в Петропавловск-камчатский, и с Петропавловска-камчатский, где-то через недельку, покажи, пожалуйста, маршруты с 15, прямой рейс до Москвы.',
 {'entities': [(45, 53, 'TRANSPORT'),
   (11, 21, 'DATE'),
   (57, 63, 'FROM'),
   (57, 63, 'LOC'),
   (57, 63, 'TO'),
   (57, 63, 'LOC'),
   (11, 21, 'DATE'),
   (45, 53, 'TRANSPORT'),
   (57, 63, 'FROM'),
   (57, 63, 'LOC'),
   (57, 63, 'TO'),
   (57, 63, 'LOC'),
   (57, 63, 'FROM'),
   (57, 63, 'LOC'),
   (11, 21, 'DATE'),
   (11, 21, 'DATE'),
   (45, 53, 'TRANSPORT'),
   (57, 63, 'TO'),
   (57, 63, 'LOC')]})

In [ ]:
print_word_indices('построй пожалуйста маршрут, самолёты на 1 мая из Ярославля в Иркутск, потом сутки ночуем и третьего числа вылетаем из Иркутска в Петропавловск-камчатский, и с Петропавловска-камчатский, где-то через недельку, покажи, пожалуйста, маршруты с 15, прямой рейс до Москвы.')

построй : 0 - 7
пожалуйста : 8 - 18
маршрут, : 19 - 26
самолёты : 28 - 36
на : 37 - 39
1 : 40 - 41
мая : 42 - 45
из : 46 - 48
Ярославля : 49 - 58
в : 59 - 60
Иркутск, : 61 - 68
потом : 70 - 75
сутки : 76 - 81
ночуем : 82 - 88
и : 89 - 90
третьего : 91 - 99
числа : 100 - 105
вылетаем : 106 - 114
из : 115 - 117
Иркутска : 118 - 126
в : 127 - 128
Петропавловск-камчатский, : 129 - 153
и : 155 - 156
с : 157 - 158
Петропавловска-камчатский, : 159 - 184
где-то : 186 - 192
через : 193 - 198
недельку, : 199 - 207
покажи, : 209 - 215
пожалуйста, : 217 - 227
маршруты : 229 - 237
с : 238 - 239
15, : 240 - 242
прямой : 244 - 250
рейс : 251 - 255
до : 256 - 258
Москвы. : 259 - 265


In [ ]:
# файл 030 whisper
("Покажи пожалуйста самолет из Перми в Тюмень на 7 февраля желательно рейсы до обеда, а из Тюмени вечерний поезд этого же числа до Перми.",
{"entities": [
    (18, 25, "TRANSPORT"),  # самолет
    (26, 28, "FROM"),  # из
    (29, 34, "LOC"),  # Перми
    (35, 36, "TO"),  # в
    (37, 43, "LOC"),  # Тюмень
    (47, 56, "DATE"),  # 7 февраля
    (57, 82, "COMFORT"),  # желательно рейсы до обеда
    (86, 88, "FROM"),  # из
    (89, 95, "LOC"),  # Тюмени
    (96, 104, "DATE"),  # вечерний
    (105, 110, "TRANSPORT"),  # поезд
    (111, 125, "DATE"),  # этого же числа
    (126, 128, "TO"),  # до
    (129, 134, "LOC"),  # Перми.
]})

('Покажи пожалуйста самолет из Перми в Тюмень на 7 февраля желательно рейсы до обеда, а из Тюмени вечерний поезд этого же числа до Перми.',
 {'entities': [(18, 25, 'TRANSPORT'),
   (26, 28, 'FROM'),
   (29, 34, 'LOC'),
   (35, 36, 'TO'),
   (37, 43, 'LOC'),
   (47, 56, 'DATE'),
   (57, 82, 'COMFORT'),
   (86, 88, 'FROM'),
   (89, 95, 'LOC'),
   (96, 104, 'DATE'),
   (105, 110, 'TRANSPORT'),
   (111, 125, 'DATE'),
   (126, 128, 'TO'),
   (129, 134, 'LOC')]})

In [ ]:
print_word_indices('Покажи пожалуйста самолет из Перми в Тюмень на 7 февраля желательно рейсы до обеда, а из Тюмени вечерний поезд этого же числа до Перми.')

Покажи : 0 - 6
пожалуйста : 7 - 17
самолет : 18 - 25
из : 26 - 28
Перми : 29 - 34
в : 35 - 36
Тюмень : 37 - 43
на : 44 - 46
7 : 47 - 48
февраля : 49 - 56
желательно : 57 - 67
рейсы : 68 - 73
до : 74 - 76
обеда, : 77 - 82
а : 84 - 85
из : 86 - 88
Тюмени : 89 - 95
вечерний : 96 - 104
поезд : 105 - 110
этого : 111 - 116
же : 117 - 119
числа : 120 - 125
до : 126 - 128
Перми. : 129 - 134


In [ ]:
# файл 031 whisper
("Подбери пожалуйста на выходной день 10 июля самолет из Южно-Сахалинска до Благовещенска и в этот же день оттуда поезд до Читы. Время отправления не важно, главное чтобы прилет был и поезд был в один день.",
{"entities": [
    (36, 43, "DATE"),  # 10 июля
    (44, 51, "TRANSPORT"),  # самолет
    (52, 54, "FROM"),  # из
    (55, 70, "LOC"),  # Южно-Сахалинска
    (71, 73, "TO"),  # до
    (74, 87, "LOC"),  # Благовещенска
    (90, 104, "DATE"),  # в этот же день
    (112, 117, "TRANSPORT"),  # поезд
    (118, 120, "TO"),  # до
    (121, 125, "LOC"),  # Читы
    (155, 203, "COMFORT"),  # главное чтобы прилет был и поезд был в один день.
]})

('Подбери пожалуйста на выходной день 10 июля самолет из Южно-Сахалинска до Благовещенска и в этот же день оттуда поезд до Читы. Время отправления не важно, главное чтобы прилет был и поезд был в один день.',
 {'entities': [(36, 43, 'DATE'),
   (44, 51, 'TRANSPORT'),
   (52, 54, 'FROM'),
   (55, 70, 'LOC'),
   (71, 73, 'TO'),
   (74, 87, 'LOC'),
   (90, 104, 'DATE'),
   (112, 117, 'TRANSPORT'),
   (118, 120, 'TO'),
   (121, 125, 'LOC'),
   (155, 203, 'COMFORT')]})

In [ ]:
print_word_indices('Подбери пожалуйста на выходной день 10 июля самолет из Южно-Сахалинска до Благовещенска и в этот же день оттуда поезд до Читы. Время отправления не важно, главное чтобы прилет был и поезд был в один день.')

Подбери : 0 - 7
пожалуйста : 8 - 18
на : 19 - 21
выходной : 22 - 30
день : 31 - 35
10 : 36 - 38
июля : 39 - 43
самолет : 44 - 51
из : 52 - 54
Южно-Сахалинска : 55 - 70
до : 71 - 73
Благовещенска : 74 - 87
и : 88 - 89
в : 90 - 91
этот : 92 - 96
же : 97 - 99
день : 100 - 104
оттуда : 105 - 111
поезд : 112 - 117
до : 118 - 120
Читы. : 121 - 125
Время : 127 - 132
отправления : 133 - 144
не : 145 - 147
важно, : 148 - 153
главное : 155 - 162
чтобы : 163 - 168
прилет : 169 - 175
был : 176 - 179
и : 180 - 181
поезд : 182 - 187
был : 188 - 191
в : 192 - 193
один : 194 - 198
день. : 199 - 203


In [ ]:
# файл 032 whisper
("покажи все самолеты вылетающие 5 сентября с Благовещенска до Красноярска.",
{"entities": [
    (11, 19, "TRANSPORT"),  # самолеты
    (31, 41, "DATE"),  # 5 сентября
    (42, 43, "FROM"),  # с
    (44, 57, "LOC"),  # Благовещенска
    (58, 60, "TO"),  # до
    (61, 72, "LOC"),  # Красноярска.
]})

('покажи все самолеты вылетающие 5 сентября с Благовещенска до Красноярска.',
 {'entities': [(11, 19, 'TRANSPORT'),
   (31, 41, 'DATE'),
   (42, 43, 'FROM'),
   (44, 57, 'LOC'),
   (58, 60, 'TO'),
   (61, 72, 'LOC')]})

In [ ]:
print_word_indices('покажи все самолеты вылетающие 5 сентября с Благовещенска до Красноярска.')

покажи : 0 - 6
все : 7 - 10
самолеты : 11 - 19
вылетающие : 20 - 30
5 : 31 - 32
сентября : 33 - 41
с : 42 - 43
Благовещенска : 44 - 57
до : 58 - 60
Красноярска. : 61 - 72


In [ ]:
# файл 033 whisper
("Срочно найди на сегодня самолёт ближайший из Хабаровска во Владивосток.",
{"entities": [
    (0, 6, "COMFORT"),  # Срочно
    (16, 23, "DATE"),  # сегодня
    (24, 31, "TRANSPORT"),  # самолет
    (42, 44, "FROM"),  # из
    (45, 55, "LOC"),  # Хабаровска
    (56, 58, "TO"),  # во
    (59, 70, "LOC"),  # Владивосток.
]})

('Срочно найди на сегодня самолёт ближайший из Хабаровска во Владивосток.',
 {'entities': [(0, 6, 'COMFORT'),
   (16, 23, 'DATE'),
   (24, 31, 'TRANSPORT'),
   (42, 44, 'FROM'),
   (45, 55, 'LOC'),
   (56, 58, 'TO'),
   (59, 70, 'LOC')]})

In [ ]:
print_word_indices('Срочно найди на сегодня самолёт ближайший из Хабаровска во Владивосток.')

Срочно : 0 - 6
найди : 7 - 12
на : 13 - 15
сегодня : 16 - 23
самолёт : 24 - 31
ближайший : 32 - 41
из : 42 - 44
Хабаровска : 45 - 55
во : 56 - 58
Владивосток. : 59 - 70


In [ ]:
# файл 034 whisper
("Надо построить маршрут из Якутска, самолёт, вылет через 3 дня, это 5 февраля, до Казани, в Казани пару дней побудем и где-то числа 11 посмотри, также, самолёт до Саратова, вылет после обеда.",
{"entities": [
    (23, 25, "FROM"),  # из
    (26, 33, "LOC"),  # Якутска
    (35, 42, "TRANSPORT"),  # самолет
    (67, 76, "DATE"),  # 5 февраля
    (78, 80, "TO"),  # до
    (81, 87, "LOC"),  # Казани
    (125, 133, "DATE"),  # числа 11
    (151, 158, "TRANSPORT"),  # самолет
    (159, 161, "TO"),  # до
    (162, 170, "LOC"),  # Саратова
    (172, 189, "COMFORT"),  # вылет после обеда.
]})

('Надо построить маршрут из Якутска, самолёт, вылет через 3 дня, это 5 февраля, до Казани, в Казани пару дней побудем и где-то числа 11 посмотри, также, самолёт до Саратова, вылет после обеда.',
 {'entities': [(23, 25, 'FROM'),
   (26, 33, 'LOC'),
   (35, 42, 'TRANSPORT'),
   (67, 76, 'DATE'),
   (78, 80, 'TO'),
   (81, 87, 'LOC'),
   (125, 133, 'DATE'),
   (151, 158, 'TRANSPORT'),
   (159, 161, 'TO'),
   (162, 170, 'LOC'),
   (172, 189, 'COMFORT')]})

In [ ]:
print_word_indices('Надо построить маршрут из Якутска, самолёт, вылет через 3 дня, это 5 февраля, до Казани, в Казани пару дней побудем и где-то числа 11 посмотри, также, самолёт до Саратова, вылет после обеда.')

Надо : 0 - 4
построить : 5 - 14
маршрут : 15 - 22
из : 23 - 25
Якутска, : 26 - 33
самолёт, : 35 - 42
вылет : 44 - 49
через : 50 - 55
3 : 56 - 57
дня, : 58 - 61
это : 63 - 66
5 : 67 - 68
февраля, : 69 - 76
до : 78 - 80
Казани, : 81 - 87
в : 89 - 90
Казани : 91 - 97
пару : 98 - 102
дней : 103 - 107
побудем : 108 - 115
и : 116 - 117
где-то : 118 - 124
числа : 125 - 130
11 : 131 - 133
посмотри, : 134 - 142
также, : 144 - 149
самолёт : 151 - 158
до : 159 - 161
Саратова, : 162 - 170
вылет : 172 - 177
после : 178 - 183
обеда. : 184 - 189


In [ ]:
# файл 035 whisper
("я с ребёнком и мужем хочу поехать с Барнаула в Ялту, новогодние праздники, выезд 28 декабря, на неделю, найди поездку суммой до 20.000.",
{"entities": [
    (0, 20, "AMOUNT"),  # я с ребёнком и мужем
    (34, 35, "TO"),  # c
    (36, 44, "LOC"),  # Барнаула
    (45, 46, "FROM"),  # в
    (47, 51, "LOC"),  # Ялту
    (81, 91, "DATE"),  # 28 декабря
    (118, 134, "BUDGET"),  # суммой до 20.000.
]})

('я с ребёнком и мужем хочу поехать с Барнаула в Ялту, новогодние праздники, выезд 28 декабря, на неделю, найди поездку суммой до 20.000.',
 {'entities': [(0, 20, 'AMOUNT'),
   (34, 35, 'TO'),
   (36, 44, 'LOC'),
   (45, 46, 'FROM'),
   (47, 51, 'LOC'),
   (81, 91, 'DATE'),
   (118, 134, 'BUDGET')]})

In [ ]:
print_word_indices('я с ребёнком и мужем хочу поехать с Барнаула в Ялту, новогодние праздники, выезд 28 декабря, на неделю, найди поездку суммой до 20.000.')

я : 0 - 1
с : 2 - 3
ребёнком : 4 - 12
и : 13 - 14
мужем : 15 - 20
хочу : 21 - 25
поехать : 26 - 33
с : 34 - 35
Барнаула : 36 - 44
в : 45 - 46
Ялту, : 47 - 51
новогодние : 53 - 63
праздники, : 64 - 73
выезд : 75 - 80
28 : 81 - 83
декабря, : 84 - 91
на : 93 - 95
неделю, : 96 - 102
найди : 104 - 109
поездку : 110 - 117
суммой : 118 - 124
до : 125 - 127
20.000. : 128 - 134


БАЗА ОБУЧЕНИЯ СУЩНОСТЕЙ, ИТОГ:

In [ ]:
TRAIN_DATA = [
    ("так еду 23 ноября и мне нужно нижнее место не у туалета, где-нибудь посередине вагона",
    {"entities": [
    (8, 17, "DATE"),  # 23 ноября
    (30, 85, "COMFORT"),  # нижнее место не у туалета, где-нибудь посередине вагона
    ]}),
    ("Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей.",
    {"entities": [
    (2, 4, "FROM"),  # из
    (5, 8, "LOC"),  # Уфы
    (16, 17, "TO"),  # в
    (18, 26, "LOC"),  # Кострому
    (44, 55, "DATE"),  # через неделю
    (64, 74, "DATE"),  # на выходные
    (106, 121, "BUDGET"),   # 10.000 руб
     ]}),
    ("Так, завтра утром мы едем, пять человек из Москвы в Петербург. Вечером мы выезжаем обратно в Москву, и через три дня мы едем в Краснодар. Также пять человек. Найди самый быстрый билет за самое быстрое время.",
    {"entities": [
    (5, 11, "DATE"),  # завтра
    (27, 39, "AMOUNT"), #пять человек
    (40, 42, "FROM"),  # из
    (43, 49, "LOC"),  # Москвы
    (50, 51, "TO"),  # в
    (52, 61, "LOC"),  # Петербург
    (63, 70, "DATE"),  # вечером
    (83, 90, "RETURN"),  # обратно
    (91, 92, "TO"),  # в
    (93, 99, "LOC"),  # Москву
    (103, 116, "DATE"),  # через три дня
    (125, 126, "TO"),  # в
    (127, 136, "LOC"),  # Краснодар
    (144, 156, "AMOUNT") # пять человек
    ]}),
    ("так не нужно самолет добраться из новосибирска до москвы я буду с двумя детьми у меня багаж две сумки по 10 килограмм найди мне билеты на 13 августа",
    {"entities": [
    (13, 20, "TRANSPORT"),  # самолёт
    (31, 33, "FROM"),  # из
    (34, 46, "LOC"),  # Новосибирска
    (47, 49, "TO"),  # до
    (50, 56, "LOC"),  # Москвы
    (57, 78, "AMOUNT"), # я буду с двумя детьми
    (86, 117, "BAGG"),    # багаж две сумки по 10 килограмм
    (138, 148, "DATE"),  # 13 августа
    ]}),
    ("поеду из Архангельска в Тверь 15 октября и 20 октября я из Твери поеду в Омск, подбери билеты",
    {"entities": [
    (6, 8, "FROM"),  # из
    (6, 21, "LOC"),  # Архангельска
    (22, 33, "FROM"),  # в
    (24, 29, "LOC"),  # Тверь
    (30, 40, "DATE"),  # 15 октября
    (43, 53, "DATE"),  # 20 октября
    (56, 58, "FROM"),  # из
    (59, 64, "LOC"),  # Твери
    (71, 72, "TO"),  # в
    (73, 77, "LOC"),  # Омск
    ]}),
   ("планирую из Новокузнецка выехать 30 декабря в Ростов, из Ростова нужно 8 января вернуться обратно, я буду с мужем и с ребёнком, найди билеты",
   {"entities": [
    (9, 11, "FROM"),  # из
    (12, 24, "LOC"),  # Новокузнецка
    (33, 43, "DATE"),  # 30 декабря
    (44, 45, "TO"),  # в
    (46, 52, "LOC"),  # Ростов
    (54, 56, "FROM"),  # из
    (57, 64, "LOC"),  # Ростова
    (71, 79, "DATE"),  # 8 января
    (90, 97, "RETURN"),  # обратно
    (99, 126, "AMOUNT"), # я буду с мужем и с ребёнком
    ]}),
   ("с 1 июля по 1 августа я поеду из Краснодара в Уфу, мне нужны билеты",
   {"entities": [
    (0, 21, "GAP"),  # с 1 июля по 1 августа
    (30, 32, "FROM"),  # из
    (33, 43, "LOC"),  # Краснодара
    (44, 45, "TO"),  # в
    (46, 49, "LOC"),  # Уфу
    ]}),
    ("так найди-ка мне билеты из Сочи в Москву, у меня есть 5.000 и мне нужен билет на сегодня",
    {"entities": [
    (24, 26, "FROM"),  # из
    (27, 31, "LOC"),  # Сочи
    (32, 33, "TO"),  # в
    (34, 40, "LOC"),  # Москву
    (54, 59, "BUDGET"),   # 5.000
    (78, 88, "DATE"),  # на сегодня
    ]}),
    ("7 января хочу на самолете долететь из Омска в Тобольск, подбери билет, я буду одна, у меня с собой ручная кладь",
    {"entities": [
    (0, 8, "DATE"),  # 7 января
    (14, 25, "TRANSPORT"),  # на самолете
    (35, 37, "FROM"),  # из
    (38, 43, "LOC"),  # Омска
    (44, 45, "TO"),  # в
    (46, 54, "LOC"),  # Тобольск
    (71, 82, "AMOUNT"), # я буду одна
    (99, 111, "BAGG"),    # ручная кладь
    ]}),
    ("с 14 по 20 февраля я поеду из Твери в Москву и потом обратно, я буду одна, подбери самый дешёвый билет",
    {"entities": [
    (0, 18, "GAP"),  # с 14 по 20 февраля
    (27, 29, "FROM"),  # из
    (30, 35, "LOC"),  # Твери
    (36, 37, "TO"),  # в
    (38, 44, "LOC"),  # Москву
    (53, 60, "RETURN"), # обратно
    (62, 73, "AMOUNT"), # я буду одна
    (75, 102, "LEVEL"), # подбери самый дешёвый билет
    ]}),
    ("хочу из Орла поехать в Пензу 20 марта и потом из Пензы мне нужно в Екатеринбург. это, наверное, число 25 марта. я буду одна, но у меня с собой будет чемодан, примерно 20 килограмм",
    {"entities": [
    (5, 7, "FROM"),  # из
    (8, 12, "LOC"),  # Орла
    (21, 22, "TO"),  # в
    (30, 35, "LOC"),  # Пензу
    (29, 37, "DATE"),  # 20 марта
    (46, 48, "FROM"),  # из
    (49, 54, "LOC"),  # Пензы
    (65, 66, "TO"),  # в
    (67, 79, "LOC"),  # Екатеринбург
    (102, 110, "DATE"),  # 25 марта
    (112, 123, "AMOUNT"), # я буду одна
    (149, 179, "BAGG"),    # чемодан, примерно 20 килограмм
    ]}),
    ("1 апреля я вылетаю из Москвы в Волгоград и из Волгограда мне нужно будет на поезде доехать до Нижневартовска, подбери билет и я буду с мужем.",
    {"entities": [
    (0, 8, "DATE"),  # 1 апреля
    (11, 18, "TRANSPORT"),  # вылетаю
    (19, 21, "FROM"),  # из
    (22, 28, "LOC"),  # Москвы
    (29, 30, "TO"),  # в
    (31, 40, "LOC"),  # Волгоград
    (43, 45, "FROM"),  # из
    (46, 56, "LOC"),  # Волгограда
    (73, 82, "TRANSPORT"),  # на поезде
    (91, 93, "TO"),  # до
    (94, 108, "LOC"),  # Нижневартовска
    (126, 140, "AMOUNT"), # я буду с мужем
    ]}),
    ("так с 1 по 9 мая я еду отдыхать в Сочи из Улан-удэ, я поеду с мужем и двумя детьми, подбери самые быстрые билеты.",
    {"entities": [
    (4, 16, "GAP"),  # с 1 по 9 мая
    (32, 33, "TO"),  # в
    (34, 38, "LOC"),  # Сочи
    (39, 41, "FROM"),  # из
    (42, 50, "LOC"),  # Улан-удэ
    (52, 82, "AMOUNT"), # я поеду с мужем и двумя детьми
    (84, 112, "LEVEL"), # подбери самые быстрые билеты
    ]}),
    ("с 3 по 17 июня я поеду из Нижневартовска в Астрахань, я буду с коллегой, нам нужно два билета туда-обратно.",
    {"entities": [
    (0, 14, "GAP"),  # с 3 по 17 июня
    (23, 25, "FROM"),  # из
    (26, 40, "LOC"),  # Нижневартовска
    (41, 42, "TO"),  # в
    (43, 52, "LOC"),  # Астрахань
    (54, 71, "AMOUNT"), # я буду с коллегой
    (83, 93, "AMOUNT"), # два билета
    (94, 106, "RETURN"), # туда-обратно
    ]}),
    ("так мне нужно из Краснодара на автобусе добраться до Сочи, посмотри пожалуйста билеты, у меня будет ручная кладь и ребёнок.",
    {"entities": [
    (14, 16, "FROM"),  # из
    (17, 27, "LOC"),  # Краснодара
    (31, 39, "TRANSPORT"),  # автобусе
    (50, 52, "TO"),  # до
    (53, 57, "LOC"),  # Сочи
    (100, 112, "BAGG"),    # ручная кладь
    (115, 122, "CHILD"),  # ребёнок
    ]}),
    ("получается 3 июля по 13 июля я поеду с ребёнком в Дагестан из Москвы, багаж будет 30 кг.",
    {"entities": [
    (11, 28, "GAP"),  # 3 июля по 13 июля
    (29, 47, "CHILD"),  # с ребёнком
    (48, 49, "TO"),  # в
    (50, 58, "LOC"),  # Дагестан
    (59, 61, "FROM"),  # из
    (62, 68, "LOC"),  # Москвы
    (70, 87, "BAGG"),    # багаж будет 30 кг
    ]}),
    ("2 августа я поеду из Астрахани в Махачкалу, подбери пожалуйста билет, у меня на поездку 7.000 руб.",
    {"entities": [
    (0, 9, "DATE"),  # 2 августа
    (18, 20, "FROM"),  # из
    (21, 30, "LOC"),  # Астрахани
    (31, 32, "TO"),  # в
    (33, 42, "LOC"),  # Махачкалу
    (54, 59, "BUDGET"),  # 7.000 руб.
    ]}),
    ("так я из Нефтекамска хочу поехать в Улан-удэ, это будет значит с 8 сентября по 19 число, подбери билет туда-обратно и у меня поедет муж, а ему нужно обязательно на поезде нижняя полочка.",
    {"entities": [
    (6, 8, "FROM"),  # из
    (9, 20, "LOC"),  # Нефтекамска
    (34, 35, "TO"),  # в
    (36, 44, "LOC"),  # Улан-удэ
    (63, 81, "GAP"),  # с 8 сентября по 19
    (103, 115, "RETURN"),  # туда-обратно
    (118, 135, "AMOUNT"),  # у меня поедет муж
    (164, 170, "TRANSPORT"),  # поезде
    (171, 185, "COMFORT"),  # нижняя полочка
    ]}),
    ("так 18 октября я поеду в командировку на Шарам, вот и я там буду неделю, и мне нужен билет, я поеду из Омска, посмотри как быстрее всего добраться.",
    {"entities": [
    (4, 14, "DATE"),  # 18 октября
    (38, 40, "TO"),  # на
    (41, 46, "LOC"),  # Шарам
    (100, 102, "FROM"),  # из
    (103, 108, "LOC"),  # Омска
    ]}),
    ("15 ноября я поеду в Нальчик. Я поеду из Томска. Найди, пожалуйста, мне билеты до 10 тысяч рублей.",
    {"entities": [
    (0, 9, "DATE"),  # 15 ноября
    (18, 19, "TO"),  # в
    (20, 27, "LOC"),  # Нальчик
    (37, 39, "FROM"),  # из
    (40, 46, "LOC"),  # Томска
    (81, 96, "BUDGET"),  # 10 тысяч рублей.
    ]}),
    ("6 декабря 6 декабря точнее А так значит по 10 декабря Я еду в Москву из Норильска Мне нужен самый быстрый билет Найди пожалуйста.",
    {"entities": [
    (0, 9, "DATE"),  # 6 декабря
    (10, 53, "GAP"),  # 6 декабря точнее А так значит по 10 декабря
    (60, 61, "TO"),  # в
    (62, 68, "LOC"),  # Москву
    (69, 71, "FROM"),  # из
    (72, 81, "LOC"),  # Норильска
    (92, 111, "COMFORT"),  # самый быстрый билет
    ]}),
    ("Покажи сегодня ближайшие рейсы до Екатеринбурга. Можно с пересадкой в Новосибирск, транзит. Только, пожалуйста, ближайшие покажи мне сегодня.",
    {"entities": [
    (7, 14, "DATE"),  # сегодня
    (31, 33, "TO"),  # до
    (34, 47, "LOC"),  # Екатеринбурга
    (68, 69, "TO"),  # в
    (20, 27, "LOC"),  # Новосибирск
    (55, 67, "TRANSIT"),  # транзит
    ]}),
    ("Поезда отменили. Покажи ближайший самолет до Кавменвод из Краснодара на завтра, 5 число утра.",
    {"entities": [
    (0, 6, "TRANSPORT"),  # Поезда
    (34, 41, "TRANSPORT"),  # самолет
    (42, 44, "TO"),  # до
    (45, 54, "LOC"),  # Кавменвод
    (55, 57, "FROM"),  # из
    (58, 68, "LOC"),  # Краснодара
    (80, 92, "DATE"),  # 5 число утра.
    ]}),
    ("Посмотри пожалуйста есть ли электрички от Тулы до Москвы на эти выходные 5-6 октября. Любое время все что есть найди покажи.",
    {"entities": [
    (28, 38, "TRANSPORT"),  # электрички
    (39, 41, "FROM"),  # от
    (42, 46, "LOC"),  # Тулы
    (47, 49, "TO"),  # до
    (50, 56, "LOC"),  # Москвы
    (73, 84, "DATE"),  # 5-6 октября
    ]}),
    ("Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть, с прибытием не позже шести утра следующего дня.",
    {"entities": [
    (14, 19, "TRANSPORT"),  # поезд
    (41, 50, "DATE"),  # 20 ноября
    (51, 53, "FROM"),  # из
    (54, 75, "LOC"),  # Комсомольска на Амуре
    (76, 78, "TO"),  # до
    (79, 89, "LOC"),  # Хабаровска
    (116, 162, "DATE"),  # с прибытием не позже шести утра следующего дня.
    ]}),
    ("покажи мне, вообще, все варианты полетов из Кавминвод до Вилюйска, неважно пересадки, не важно время, главное чтобы вылет был послезавтра 7 июля.",
    {"entities": [
    (33, 40, "TRANSPORT"),  # полетов
    (41, 43, "FROM"),  # из
    (44, 53, "LOC"),  # Кавминвод
    (54, 56, "TO"),  # до
    (57, 65, "LOC"),  # Вилюйска
    (138, 144, "DATE"),  # 7 июля.
    ]}),
    ("покажи мне на декабрь, к примеру, 5 декабря, самолеты из Москвы до Петропавловска-Камчатский интересные варианты, которые будут прямые, без пересадки, обратно пока не показывай.",
    {"entities": [
    (11, 21, "DATE"),  # на декабрь
    (34, 43, "DATE"),  # 5 декабря
    (45, 53, "TRANSPORT"),  # самолеты
    (54, 56, "FROM"),  # из
    (57, 63, "LOC"),  # Москвы
    (64, 66, "TO"),  # до
    (67, 92, "LOC"),  # Петропавловска-Камчатский
    (128, 149, "TRANZIT"),  # прямые, без пересадки
    (151, 176, "RETURN"),  # обратно пока не показывай.
    ]}),
    ("срочно, на завтра, поезд утренний, Тула  Казань вообще все варианты, неважно какая цена, главное чтобы утром.",
    {"entities": [
    (11, 21, "DATE"),  # завтра
    (45, 53, "TRANSPORT"),  # поезд
    (11, 21, "DATE"),  # утренний
    (57, 63, "LOC"),  # Тула
    (57, 63, "LOC"),  # Казань
    (81, 96, "BUDGET"),  # неважно какая цена
    (34, 43, "COMFORT"),  # утром
    ]}),
    ("построй пожалуйста маршрут, самолёты на 1 мая из Ярославля в Иркутск, потом сутки ночуем и третьего числа вылетаем из Иркутска в Петропавловск-камчатский, и с Петропавловска-камчатский, где-то через недельку, покажи, пожалуйста, маршруты с 15, прямой рейс до Москвы.",
    {"entities": [
    (45, 53, "TRANSPORT"),  # самолёты
    (11, 21, "DATE"),  # 1 мая
    (57, 63, "FROM"),  # из
    (57, 63, "LOC"),  # Ярославля
    (57, 63, "TO"),  # в
    (57, 63, "LOC"),  # Иркутск
    (11, 21, "DATE"),  # третьего числа
    (45, 53, "TRANSPORT"),  # вылетаем
    (57, 63, "FROM"),  # из
    (57, 63, "LOC"),  # Иркутска
    (57, 63, "TO"),  # в
    (57, 63, "LOC"),  # Петропавловск-камчатский
    (57, 63, "FROM"),  # из
    (57, 63, "LOC"),  # Петропавловск-камчатский
    (11, 21, "DATE"),  # через недельку
    (11, 21, "DATE"),  # с 15
    (45, 53, "TRANSPORT"),  # прямой рейс
    (57, 63, "TO"),  # до
    (57, 63, "LOC"),  # Москвы.
    ]}),
    ("Покажи пожалуйста самолет из Перми в Тюмень на 7 февраля желательно рейсы до обеда, а из Тюмени вечерний поезд этого же числа до Перми.",
    {"entities": [
    (18, 25, "TRANSPORT"),  # самолет
    (26, 28, "FROM"),  # из
    (29, 34, "LOC"),  # Перми
    (35, 36, "TO"),  # в
    (37, 43, "LOC"),  # Тюмень
    (47, 56, "DATE"),  # 7 февраля
    (57, 82, "COMFORT"),  # желательно рейсы до обеда
    (86, 88, "FROM"),  # из
    (89, 95, "LOC"),  # Тюмени
    (96, 104, "DATE"),  # вечерний
    (105, 110, "TRANSPORT"),  # поезд
    (111, 125, "DATE"),  # этого же числа
    (126, 128, "TO"),  # до
    (129, 134, "LOC"),  # Перми.
    ]}),
    ("Подбери пожалуйста на выходной день 10 июля самолет из Южно-Сахалинска до Благовещенска и в этот же день оттуда поезд до Читы. Время отправления не важно, главное чтобы прилет был и поезд был в один день.",
    {"entities": [
    (36, 43, "DATE"),  # 10 июля
    (44, 51, "TRANSPORT"),  # самолет
    (52, 54, "FROM"),  # из
    (55, 70, "LOC"),  # Южно-Сахалинска
    (71, 73, "TO"),  # до
    (74, 87, "LOC"),  # Благовещенска
    (90, 104, "DATE"),  # в этот же день
    (112, 117, "TRANSPORT"),  # поезд
    (118, 120, "TO"),  # до
    (121, 125, "LOC"),  # Читы
    (155, 203, "COMFORT"),  # главное чтобы прилет был и поезд был в один день.
    ]}),
    ("покажи все самолеты вылетающие 5 сентября с Благовещенска до Красноярска.",
    {"entities": [
    (11, 19, "TRANSPORT"),  # самолеты
    (31, 41, "DATE"),  # 5 сентября
    (42, 43, "FROM"),  # с
    (44, 57, "LOC"),  # Благовещенска
    (58, 60, "TO"),  # до
    (61, 72, "LOC"),  # Красноярска.
    ]}),
    ("Срочно найди на сегодня самолёт ближайший из Хабаровска во Владивосток.",
    {"entities": [
    (0, 6, "COMFORT"),  # Срочно
    (16, 23, "DATE"),  # сегодня
    (24, 31, "TRANSPORT"),  # самолет
    (42, 44, "FROM"),  # из
    (45, 55, "LOC"),  # Хабаровска
    (56, 58, "TO"),  # во
    (59, 70, "LOC"),  # Владивосток.
    ]}),
    ("Надо построить маршрут из Якутска, самолёт, вылет через 3 дня, это 5 февраля, до Казани, в Казани пару дней побудем и где-то числа 11 посмотри, также, самолёт до Саратова, вылет после обеда.",
    {"entities": [
    (23, 25, "FROM"),  # из
    (26, 33, "LOC"),  # Якутска
    (35, 42, "TRANSPORT"),  # самолет
    (67, 76, "DATE"),  # 5 февраля
    (78, 80, "TO"),  # до
    (81, 87, "LOC"),  # Казани
    (125, 133, "DATE"),  # числа 11
    (151, 158, "TRANSPORT"),  # самолет
    (159, 161, "TO"),  # до
    (162, 170, "LOC"),  # Саратова
    (172, 189, "COMFORT"),  # вылет после обеда.
    ]}),
    ("я с ребёнком и мужем хочу поехать с Барнаула в Ялту, новогодние праздники, выезд 28 декабря, на неделю, найди поездку суммой до 20.000.",
    {"entities": [
    (0, 20, "AMOUNT"),  # я с ребёнком и мужем
    (34, 35, "TO"),  # c
    (36, 44, "LOC"),  # Барнаула
    (45, 46, "FROM"),  # в
    (47, 51, "LOC"),  # Ялту
    (81, 91, "DATE"),  # 28 декабря
    (118, 134, "BUDGET"),  # суммой до 20.000.
    ]})
]

 ОБУЧЕНИЕ МОДЕЛИ

ЗАГРУЗКА ПРЕДОБУЧЕННОЙ МОДЕЛИ И ЗАЯВКА НОВЫХ СУЩНОСТЕЙ С ДАТАСЕТА ОБУЧЕНИЯ

In [ ]:
nlp = spacy.load("ru_core_news_sm")
ner = nlp.get_pipe("ner")
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
ner = nlp.get_pipe("ner")
print("Обученные + наши сущности:", ner.labels)


Обученные + наши сущности: ('AMOUNT', 'BAGG', 'BUDGET', 'CHILD', 'COMFORT', 'DATE', 'FROM', 'GAP', 'LEVEL', 'LOC', 'ORG', 'PER', 'RETURN', 'TO', 'TRANSIT', 'TRANSPORT', 'TRANZIT')


ФУНКЦИЯ ДООБУЧЕНИЯ ПРЕДОБУЧЕННОЙ МОДЕЛИ НА НОВЫХ СУЩНОСТЯХ

In [ ]:
import spacy
from spacy.training import offsets_to_biluo_tags

nlp = spacy.load("en_core_web_sm")

def analyze_entity_alignment(text, entities):
    doc = nlp.make_doc(text)
    biluo_tags = offsets_to_biluo_tags(doc, entities)
    for token, tag in zip(doc, biluo_tags):
        print(f"{token.text}: {tag}")

# Пример текста и меток сущностей
text = "Покажи сегодня ближайшие рейсы до Екатеринбурга."
entities = [(7, 14, 'DATE'), (31, 33, 'TO'), (34, 47, 'LOC')]

# Анализ выравнивания сущностей
analyze_entity_alignment(text, entities)


Покажи: O
сегодня: U-DATE
ближайшие: O
рейсы: O
до: U-TO
Екатеринбурга: U-LOC
.: O


In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
   optimizer = nlp.begin_training()
      for itn in range(50):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, losses=losses)
        print(losses)

In [ ]:
nlp.to_disk("/content/drive/MyDrive/NLP_spacy")

 РАСПОЗНОВАНИЕ СУЩНОСТЕЙ

ЗАГРУЖАЕМ МОДЕЛЬ

In [ ]:
work_path = '/content/drive/MyDrive/NLP_spacy'
output_dir = work_path
if not os.path.isdir(output_dir): print('Папка', output_dir, 'не найдена!')
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

Loading from /content/drive/MyDrive/NLP_spacy


ТЕСТИРУЕМ РАБОТУ МОДЕЛИ РАСПОЗНАВАНИЯ СУЩНОСТЕЙ

In [ ]:
# тестирование инициализированной модели
#test_text = "Самолет до Москвы из Владивостока. Там нужно быть 5 мая вечером. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Можно даже через Новосибирск. Обратно 26 числа. Покажи все, что есть. Общий бюджет 10000 рублей."
test_text = "Я из Уфы, поеду в Кострому. Мне нужен билет через неделю. Лучше на выходные. Так, у меня будет на поездку 10 тысяч рублей."
#test_text = "Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть. С прибытием не позже 6 утра следующего дня."
#test_text = "Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря пассажир 1."

doc = nlp(test_text)

print([(w.text, w.pos_) for w in doc])
print()
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
print()
spacy.displacy.render(doc, style="ent", jupyter=True)

[('Я', 'PROPN'), ('из', 'PROPN'), ('Уфы', 'PROPN'), (',', 'PUNCT'), ('поеду', 'VERB'), ('в', 'PROPN'), ('Кострому', 'PROPN'), ('.', 'PUNCT'), ('Мне', 'PROPN'), ('нужен', 'PROPN'), ('билет', 'ADJ'), ('через', 'ADV'), ('неделю', 'ADV'), ('.', 'PUNCT'), ('Лучше', 'PROPN'), ('на', 'PROPN'), ('выходные', 'PROPN'), ('.', 'PUNCT'), ('Так', 'PROPN'), (',', 'PUNCT'), ('у', 'DET'), ('меня', 'NOUN'), ('будет', 'PROPN'), ('на', 'PROPN'), ('поездку', 'VERB'), ('10', 'NUM'), ('тысяч', 'ADJ'), ('рублей', 'NOUN'), ('.', 'PUNCT')]

Entities [('Кострому', 'PERSON'), ('Мне', 'PERSON'), ('10', 'CARDINAL')]



In [ ]:
# тестирование инициализированной модели
#test_text = "Самолет до Москвы из Владивостока. Там нужно быть 5 мая вечером. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Можно даже через Новосибирск. Обратно 26 числа. Покажи все, что есть. Общий бюджет 10000 рублей."
test_text = "Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей."
#test_text = "Срочно покажи поезд ближайший на сегодня 20 ноября из Комсомольска на Амуре до Хабаровска, ближайший, который есть. С прибытием не позже 6 утра следующего дня."
#test_text = "Какие есть варианты добраться 15 декабря из Краснодара в Новосибирск? Обратный возврат 19 декабря пассажир 1."

doc = nlp(test_text)

print([(w.text, w.pos_) for w in doc])
print()
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
print()
spacy.displacy.render(doc, style="ent", jupyter=True)

[('Самолет', 'INTJ'), ('до', 'ADV'), ('Москвы', 'PROPN'), ('из', 'PROPN'), ('Владивостока', 'PROPN'), ('на', 'PROPN'), ('5', 'NUM'), ('мая', 'NOUN'), ('.', 'PUNCT'), ('Какие', 'PROPN'), ('есть', 'NOUN'), ('варианты', 'NOUN'), ('?', 'PUNCT'), ('На', 'PROPN'), ('двоих', 'NOUN'), ('пассажиров', 'PROPN'), ('без', 'PROPN'), ('детей', 'PROPN'), ('.', 'PUNCT'), ('Без', 'PROPN'), ('багажа', 'PROPN'), ('.', 'PUNCT'), ('Без', 'PROPN'), ('пересадок', 'NOUN'), ('.', 'PUNCT'), ('Ну', 'NOUN'), ('и', 'ADJ'), ('потом', 'NOUN'), ('обратно', 'NOUN'), ('.', 'PUNCT'), ('Где', 'PROPN'), ('-', 'PUNCT'), ('то', 'PROPN'), ('20', 'NUM'), ('мая', 'NOUN'), ('.', 'PUNCT'), ('Покажи', 'PROPN'), ('все', 'NOUN'), (',', 'PUNCT'), ('что', 'NOUN'), ('есть', 'NOUN'), ('.', 'PUNCT'), ('Общий', 'PROPN'), ('бюджет', 'VERB'), ('10000', 'NUM'), ('рублей', 'NOUN'), ('.', 'PUNCT')]

Entities [('5', 'CARDINAL'), ('Какие', 'ORG'), ('двоих пассажиров', 'PERSON'), ('детей', 'DATE'), ('20', 'CARDINAL'), ('Общий', 'ORG'), ('10000', 

In [ ]:
entities = [{'text': ent.text, 'lemma': ent.lemma_, 'label': ent.label_} for ent in doc.ents]
entities

[{'text': '5', 'lemma': '5', 'label': 'CARDINAL'},
 {'text': 'Какие', 'lemma': 'Какие', 'label': 'ORG'},
 {'text': 'двоих пассажиров', 'lemma': 'двоих пассажиров', 'label': 'PERSON'},
 {'text': 'детей', 'lemma': 'детей', 'label': 'DATE'},
 {'text': '20', 'lemma': '20', 'label': 'CARDINAL'},
 {'text': 'Общий', 'lemma': 'Общий', 'label': 'ORG'},
 {'text': '10000', 'lemma': '10000', 'label': 'CARDINAL'}]

ФОРМИРОВАНИЕ JSON ФАЙЛА

ОБРАБАТЫВАЕМ ГОРОДА ПО ПОРОГУ СХОДСТВА

In [ ]:
def load_city_list(file_path):  # Загружам список городов из текстового файла.
   with open(file_path, 'r', encoding='utf-8') as f:
        cities = [line.strip() for line in f]
   return cities

def find_closest_city(city, city_list):   # Находим ближайший город в списке с использованием метода наибольшего общего подпоследовательности.
    matches = difflib.get_close_matches(city, city_list, n=1, cutoff=0.8)   # n - число возвращаемых совпадений, cutoff -  минимальный порог сходства
    # get_close_matches - возвращает список ближайших совпадений, найденных в city_list
    # n - число возвращаемых совпадений, cutoff -  минимальный порог сходства
    closest_city = matches[0] if len(matches) > 0 else None
    return closest_city

def correct_city(city, city_list):    # Обработка списока городов и проверка на соответствие.
    closest_city = find_closest_city(city, city_list)
    if city == "Питер" or city == "Ленинград":        # Выявленная замена
        closest_city = "Санкт-Петербург"
    if city == "Минводы" or city == "Кавминводы"  or city == "Каменвода" or city == "Каменводы":     # Выявленная замена
        closest_city = "Минеральные Воды"
    if city == "Нижний Арктовского" or city == "Нижний Арктовский":     # Выявленная замена
        closest_city = "Нижневартовск"
    return closest_city

file_path = "/content/drive/MyDrive/Audio/cities.txt"
all_cities = load_city_list(file_path)


ОБРАБАТЫВАЕМ ЦИФРЫ

In [ ]:
# служебные функции "добавить" и "умножить"
def add(count): return lambda x: x+count
def mul(count): return lambda x: x*count if x > 0 else count

# словарь вариантов токенов
startTokens = {
    "один": add(1), "одна": add(1), "одного": add(1),
    "два": add(2),  "две": add(2), "двое": add(2), "вдвоём": add(2),
    "три": add(3), "трое": add(3), "втроём": add(3),
    "четыре": add(4), "четверо": add(4), "вчетвером": add(4),
    "пять": add(5), "пятеро": add(5), "впятером": add(5),
    "шесть": add(6), "шестеро": add(6), "вшестером": add(6),
    "семь": add(7), "семеро": add(7),
    "восемь": add(8),
    "девять": add(9),

    "десяток": mul(10), "десятка": mul(10), "десятков": mul(10),

    "десять": add(10),
    "одиннадцать": add(11),
    "двенадцать": add(12),
    "тринадцать": add(13),
    "четырнадцать": add(14),
    "пятнадцать": add(15),
    "шестнадцать": add(16),
    "семнадцать": add(17),
    "восемнадцать": add(18),
    "девятнадцать": add(19),
    "двадцать": add(20),
    "тридцать": add(30),
    "сорок": add(40),
    "пятьдесят": add(50),
    "шестьдесят": add(60),
    "семьдесят": add(70),
    "восемьдесят": add(80),
    "девяносто": add(90),

    "сотня": mul(100), "сотни": mul(100), "сотен": mul(100),

    "сто": add(100),
    "двести": add(200),
    "триста": add(300),
    "четыреста": add(400),
    "пятьсот": add(500),
    "шестьсот": add(600),
    "семьсот": add(700),
    "восемьсот": add(800),
    "девятьсот": add(900),

    "тысяча": mul(1000), "тысячи": mul(1000), "тысяч": mul(1000), "тыс": mul(1000)
}

# функция определения действия по токену
def matchToken(token):
  try:
    return startTokens[token]
  except:
    if token.isdigit():
      return add(int(token))
    return add(0)


# функция для преобразования числа, заданного словами, в число
def str2num(st):

    if st.isdigit():
      return int(st)

    tokens = st.split(" ")
    value = 0
    for token in tokens:
      value = matchToken(token)(value)

    return value


ФУНКЦИЯ ФОРМИРОВАНИЯ НОВОЙ ПОЕЗДКИ

In [ ]:
def get_new_route():

    route = {
        'city_from': None,     # город отправления (может быть null)
        'date_from': None,     # дата отправления (может быть null)
        'date_from_t': None,   # дата отправления, если не удалось определить точное значение (может быть null)
        'time_from': None,     # время отправления (может быть null)
        'city_to': None,       # город прибытия (может быть null)
        'date_to': None,       # дата прибытия (может быть null)
        'date_to_t': None,     # дата прибытия, если не удалось определить точное значение (может быть null)
        'time_to': None,       # время прибытия (может быть null)
        'amount': None,        # количество пассажиров (минимум один)
        'transit': None,       # количество пересадок (может быть null)
#        'replace_city': None,  # города для возможных пересадок (может быть null)
        'transport': None,     # вид транспорта -  'поезд', 'самолет', 'автобус', 'паром' (может быть null)
        'company': None,       # компания-перевозчик (может быть null)
        'animal': None,        # с животными (может быть null)
        'children': None,      # количество детей (может быть null)
        'baggage': None,       # с багажом (может быть null)
        'level': None,         # класс обслуживания - 'бизнес', 'эконом', 'купе', 'плацкарт' (может быть null)
        'comfort': None,       # список дополнительных пожеланий - 'нижняя полка', 'мягкий вагон', 'с питанием', 'женское купе', 'не у туалета', 'биотуалет'  (может быть null)
        'insurance': None      # страховка (может быть null)
    }

    return route


ФУНКЦИЯ ФОРМИРОВАНИЯ ДЖЕЙСОН ФАЙЛА

In [ ]:
def create_json(doc, filename, transcribator='whisper(model=large)'):

    entities = [{'text': ent.text, 'lemma': ent.lemma_, 'label': ent.label_} for ent in doc.ents]

    json_data = {
        'route': [],                      # маршрут - список поездок (обязательно должен быть хотя бы один элемент)
        'all_budget': None,               # общий бюджет на весь маршрут (может быть null)
        'filename': filename,             # название транскрибированного аудиофайла
#        'real_text': None,
        'transcrib_text': doc.text,       # транскрибированный текст
        'transcribator': transcribator    # использованный транскрибатор
    }

    # настройки для парсера дат
    settings = {
        'DATE_ORDER': 'DMY',
        'PREFER_DATES_FROM': 'future'
    }

    #format_date = '%d.%m.%Y'
    format_date = '%d.%m'

    route = get_new_route()  # создаем новую поездку
    pred_route = route.copy()  # нужно определить и предыдущую поездку

    new_route = True
    ll = len(entities)
    for i in range(ll):
        entity = entities[i]
        if entity['label'] == 'LOC':  # анализ сущности LOCATION
            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if (entity_pred['label'] == 'FROM'):  # если предыдущая сущность FROM, то это город отправления
                route['city_from'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
            elif (entity_pred['label'] == 'TO'):  # если предыдущая сущность TO, то это город назначения
                route['city_to'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
            elif entity_pred['label'] == 'TRANSIT':  # если предыдущая сущность TRANSIT, то это город пересадки
                if new_route:
               #     route['replace_city'] = []
                    new_route = False
                # if route['replace_city'] is None:
                #     route['replace_city'] = []
                # route['replace_city'].append(entity['lemma'].title().replace('.', ''))
                route['transit'] = 1
            elif route['city_from'] is None:  # если предыдущая сущность не подходящая, город отправления определен, а город назначения еще не определен, то это город назначения
                route['city_from'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
            elif route['city_to'] is None:  # если предыдущая сущность не подходящая и город отправления еще не определен, то это город отправления
                route['city_to'] = correct_city(entity['lemma'].title().replace('.', ''), all_cities)
        elif entity['label'] == 'DATE':  # анализ сущности DATE
            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if (entity_pred['label'] == 'ARRIVAL'):  # проверим предыдущую сущность, если она ARRIVAL, то это дата прибытия
                st = entity['lemma']
                if st.find('числ') > -1:  # если найдем "число", то заменим на месяц от текущей даты
                    st = st.replace('число', datetime.strftime(date.today(), '%B'))
                dt = dateparser.parse(st, settings=settings)
                if dt is None: dt = date.today()
                route['date_to'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_to_t'] = entity['text']
            elif route['date_from_t'] is None:  # записываем в дату отправления, если она еще не задана
                st = entity['lemma']
                if st.find('числ') > -1:  # если найдем "число", то заменим на текущий месяц
                    if pred_route['date_from'] is not None:  # если была предыдущая поездка и определена дата отправления
                        st = st.replace('число', datetime.strptime(pred_route['date_from'], format_date).strftime('%B'))
                    elif pred_route['date_to'] is not None:  # ... или определена дата прибытия
                        st = st.replace('число', datetime.strptime(pred_route['date_to'], format_date).strftime('%B'))
                    else:  # иначе возьмем месяц от текущей даты
                        st = st.replace('число', datetime.strftime(date.today(), '%B'))
                dt = dateparser.parse(st, settings=settings)
                if dt is None: dt = date.today()
                route['date_from'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_from_t'] = entity['text']
            else:  # а если дата отправления уже определена, то записываем в дату прибытия
                st = entity['lemma']
                if st.find('числ') > -1:  # если найдем "число", то заменим на текущий месяц
                    if route['date_from'] is not None:  # если определена дата отправления
                        st = st.replace('число', datetime.strptime(route['date_from'], format_date).strftime('%B'))
                    else:  # иначе возьмем месяц от текущей даты
                        st = st.replace('число', datetime.strftime(date.today(), '%B'))
                dt = dateparser.parse(st, settings=settings)
                if dt is None: dt = date.today()
                route['date_to'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_to_t'] = entity['text']
        elif entity['label'] == 'GAP':  # анализ сущности GAP
            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if i < ll-1:
                entity_next = entities[i+1]
            else:
                entity_next = entity
            if (entity_pred['label'] == 'RETURN') or (entity_next['label'] == 'RETURN'):
                # проверим предыдущую и следующую сущность и, если она RETURN, то анализ GAP пропускаем - будет выполняться при анализе RETURN
                continue
            elif route['date_from_t'] is None:  # записываем в дату отправления, если она еще не задана
                dt = dateparser.parse(entity['text'], settings=settings)
                if dt is None: dt = date.today()
                route['date_from'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_from_t'] = entity['text']
            else:  # а если дата отправления уже определена, то записываем в дату прибытия
                dt = dateparser.parse(entity['text'], settings=settings)
                if dt is None: dt = date.today()
                route['date_to'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая
                route['date_to_t'] = entity['text']
        elif entity['label'] == 'TIME':  # анализ сущности TIME
            if route['time_from'] is None:  # проверим время отправления, если оно еще не задано
                # дополнительно проверим даты отправления и прибытия
                # если дата отправления не задана, а дата прибытия определена, то скорее всего это время прибытия
                if (route['date_from_t'] is None) and (route['date_to_t'] is not None):
                    route['time_to'] = entity['text']
                else:  # в других случаях - время отправления
                    route['time_from'] = entity['text']
            else:  # а если время отправления уже определено, то записываем во время прибытия
                route['time_to'] = entity['text']
        elif entity['label'] == 'TRANSIT':  # здесь отлавливаем "без пересадок"
            if entity['text'].find('без') == -1: route['transit'] = 0
        elif entity['label'] == 'AMOUNT':  # кол-во пассажиров
            route['amount'] = str2num(entity['lemma'].replace('.', ''))
        elif entity['label'] == 'TRANSPORT':  # добавляем вид транспорта
            if route['transport'] is None: route['transport'] = []
            route['transport'].append(entity['lemma'].replace('.', ''))
        elif entity['label'] == 'ORG':  # добавляем вид организацию-перевозчика
            if route['company'] is None: route['company'] = []
            route['company'].append(entity['lemma'].title().replace('.', ''))
        elif entity['label'] == 'ANIMAL':  # "без животных" или есть какие-то животные
            route['animal'] = entity['text'].find('без') == -1
        elif entity['label'] == 'CHILD':  # кол-во детей, с проверкой на "без детей"
            if entity['text'].find('без') > -1:
                route['children'] = 0
            else:
                col = str2num(entity['lemma'].replace('.', ''))
                if col == 0: col = 1
                route['children'] = col
        elif entity['label'] == 'BAGG':  # "без багажа" или есть какой-то багаж
            route['baggage'] = entity['text'].find('без') == -1
        elif entity['label'] == 'LEVEL':  # добавляем уровень обслуживания
            if route['level'] is None: route['level'] = []
            route['level'].append(entity['text'])
        elif entity['label'] == 'COMFORT':  # добавляем пожелания к комфорту
            if route['comfort'] is None: route['comfort'] = []
            route['comfort'].append(entity['text'])
        elif entity['label'] == 'INSUR':  # "без страховки" или страховка есть
            route['insurance'] = entity['text'].find('без') == -1
        elif entity['label'] == 'BUDGET':  # общий бюджет
            json_data['all_budget'] = str2num(entity['lemma'].replace('.', ''))
        elif entity['label'] == 'RETURN':
            # если встречаем сущность RETURN, то формируем новую поездку, в которую записываем города отправления и прибытия из предыдущей поездки, но в обратном порядке
            # все остальные сущности просто копируем, предполагая, что все ранее определенные особенности поездки будут такими же
            # при этом даты не копируем, т.к. ясно, что обратная поездка будет позже
            # если дальше при разборе фразы будут найдены сущности характеристик поездки, то просто перезапишем
            # особенность с транзитными городами: если находим сущность TRANSIT, то обнуляем транзитные города предыдущей поездки

            if i > 0:
                entity_pred = entities[i-1]
            else:
                entity_pred = entity
            if entity_pred['label'] == 'RETURN': continue  # не обрабатываем, если несколько сущностей RETURN идет подряд

            if (route['date_from_t'] is None) and (route['date_to_t'] is None):  # если дата отправления осталась не опеределенной, то заполняем ее текущей датой
                route['date_from'] = datetime.strftime(date.today(), format_date)
                route['date_from_t'] = ''

            pred_route = route.copy()  # скопируем текущую поездку
            json_data['route'].append(route)  # добавляем текущую поездку в маршрут
            route = get_new_route()  # создаем новую поездку
            new_route = True
            # заполним некоторые параметры новой поездки на основании предыдущей
            route['city_from'] = pred_route['city_to']
            route['city_to'] = pred_route['city_from']
            route['transit'] = pred_route['transit']
            #route['replace_city'] = pred_route['replace_city']
            route['amount'] = pred_route['amount']
            route['transport'] = pred_route['transport']
            route['animal'] = pred_route['animal']
            route['children'] = pred_route['children']
            route['baggage'] = pred_route['baggage']
            route['level'] = pred_route['level']
            route['comfort'] = pred_route['comfort']
            route['insurance'] = pred_route['insurance']

            if i < ll-1:
                entity_next = entities[i+1]
            else:
                entity_next = entity
            if (entity_pred['label'] == 'GAP') or (entity_next['label'] == 'GAP'):  # проверим предыдущую и следующую сущность, если она GAP, то попробуем определить дату отправления для новой поездки
                if entity_pred['label'] == 'GAP':
                    route['date_from_t'] = entity_pred['text']
                else:
                    route['date_from_t'] = entity_next['text']
                settings_modify = settings.copy()
                # в настройки парсера дат добавляем "считать от даты..."
                if pred_route['date_from'] is not None: # если определена дата отправления
                    settings_modify['RELATIVE_BASE'] = datetime.strptime(pred_route['date_from'], format_date)
                elif pred_route['date_to'] is not None: # если определена дата прибытия
                    settings_modify['RELATIVE_BASE'] = datetime.strptime(pred_route['date_to'], format_date)
                # или останется текущая дата

                dt = dateparser.parse(route['date_from_t'], settings=settings_modify)
                if dt is None: dt = date.today()
                route['date_from'] = datetime.strftime(dt, format_date)  # если парсер не сможет определить дату, то будет текущая

    json_data['route'].append(route)  # добавляем текущую поездку в маршрут

    return json_data


ЗАПУСК ПИПЛАЙНА

In [ ]:
filename = '/content/drive/MyDrive/Audio/extracted1/whisper1/002.txt'
create_json(doc, filename)

{'route': [{'city_from': None,
   'date_from': '25.03',
   'date_from_t': 'детей',
   'time_from': None,
   'city_to': None,
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': ['Какие', 'Общий'],
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': '/content/drive/MyDrive/Audio/extracted1/whisper1/002.txt',
 'transcrib_text': 'Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей.',
 'transcribator': 'whisper(model=large)'}

In [ ]:
filename = '/content/drive/MyDrive/Audio/extracted1/whisper1/003.txt'
create_json(doc, filename)

{'route': [{'city_from': None,
   'date_from': '25.03',
   'date_from_t': 'детей',
   'time_from': None,
   'city_to': None,
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': ['Какие', 'Общий'],
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': '/content/drive/MyDrive/Audio/extracted1/whisper1/003.txt',
 'transcrib_text': 'Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей.',
 'transcribator': 'whisper(model=large)'}

In [ ]:
filename = '/content/drive/MyDrive/Audio/extracted1/whisper1/010.txt'
create_json(doc, filename)

{'route': [{'city_from': None,
   'date_from': '25.03',
   'date_from_t': 'детей',
   'time_from': None,
   'city_to': None,
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': ['Какие', 'Общий'],
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': '/content/drive/MyDrive/Audio/extracted1/whisper1/010.txt',
 'transcrib_text': 'Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей.',
 'transcribator': 'whisper(model=large)'}

In [ ]:
filename = '/content/drive/MyDrive/Audio/extracted1/whisper1/006.txt'
create_json(doc, filename)

{'route': [{'city_from': None,
   'date_from': '25.03',
   'date_from_t': 'детей',
   'time_from': None,
   'city_to': None,
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': ['Какие', 'Общий'],
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': '/content/drive/MyDrive/Audio/extracted1/whisper1/006.txt',
 'transcrib_text': 'Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей.',
 'transcribator': 'whisper(model=large)'}

In [ ]:
filename = '/content/drive/MyDrive/Audio/extracted1/whisper1/026.txt'
create_json(doc, filename)

{'route': [{'city_from': None,
   'date_from': '25.03',
   'date_from_t': 'детей',
   'time_from': None,
   'city_to': None,
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': ['Какие', 'Общий'],
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': '/content/drive/MyDrive/Audio/extracted1/whisper1/026.txt',
 'transcrib_text': 'Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей.',
 'transcribator': 'whisper(model=large)'}

In [ ]:
filename = '/content/drive/MyDrive/Audio/extracted1/whisper1/041.txt'
create_json(doc, filename)

{'route': [{'city_from': None,
   'date_from': '25.03',
   'date_from_t': 'детей',
   'time_from': None,
   'city_to': None,
   'date_to': None,
   'date_to_t': None,
   'time_to': None,
   'amount': None,
   'transit': None,
   'transport': None,
   'company': ['Какие', 'Общий'],
   'animal': None,
   'children': None,
   'baggage': None,
   'level': None,
   'comfort': None,
   'insurance': None}],
 'all_budget': None,
 'filename': '/content/drive/MyDrive/Audio/extracted1/whisper1/041.txt',
 'transcrib_text': 'Самолет до Москвы из Владивостока на 5 мая. Какие есть варианты? На двоих пассажиров без детей. Без багажа. Без пересадок. Ну и потом обратно. Где-то 20 мая. Покажи все, что есть. Общий бюджет 10000 рублей.',
 'transcribator': 'whisper(model=large)'}